<a href="https://colab.research.google.com/github/sunshuofeng/AIssf/blob/master/reid_model%2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 数据和包的准备

In [1]:
!pip install geffnet
!pip install git+https://github.com/pabloppp/pytorch-tools -U
!pip install thop 
!pip install -U people_segmentation
!pip install lambda-networks
!pip install kaggle
!pip install iglovikov_helper_functions

  Cloning https://github.com/pabloppp/pytorch-tools to /tmp/pip-req-build-ht7by5ol
  Running command git clone -q https://github.com/pabloppp/pytorch-tools /tmp/pip-req-build-ht7by5ol
  Created wheel for torchtools: filename=torchtools-0.2.6-cp37-none-any.whl size=21149 sha256=cd4602523e5761d505035195e91dc067db44be0dbfec4b5ecb8183c238e4e423
  Stored in directory: /tmp/pip-ephem-wheel-cache-ii2orpv0/wheels/53/7a/b0/86ffb126404564b151069f4d8d9f43dbfe3f17bfc0b898b45f
Successfully built torchtools
  Found existing installation: torchtools 0.2.6
    Uninstalling torchtools-0.2.6:
      Successfully uninstalled torchtools-0.2.6
Requirement already up-to-date: people_segmentation in /usr/local/lib/python3.7/dist-packages (0.0.4)


In [2]:
import json
token = {"username":"ssfailearning","key":"bccedfffce653f07d5acaf4409341c47"}
with open('/content/kaggle.json', 'w') as file:
  json.dump(token, file)
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle config set -n path -v /content
!kaggle datasets download -d rayiooo/reid_market-1501
!kaggle datasets download -d ssf0921/p-final-reid
!kaggle datasets download -d ssfailearning/vc-cloths

- path is now set to: /content
reid_market-1501.zip: Skipping, found more recently modified local copy (use --force to force download)
p-final-reid.zip: Skipping, found more recently modified local copy (use --force to force download)
vc-cloths.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
import sys
!git clone https://github.com/Linzaer/Ultra-Light-Fast-Generic-Face-Detector-1MB.git
sys.path.append('Ultra-Light-Fast-Generic-Face-Detector-1MB')

fatal: destination path 'Ultra-Light-Fast-Generic-Face-Detector-1MB' already exists and is not an empty directory.


In [4]:
import os
import zipfile

zip_file = zipfile.ZipFile('/content/datasets/rayiooo/reid_market-1501/reid_market-1501.zip')
for names in zip_file.namelist():
        zip_file.extract(names,'datasets')
zip_file.close()

zip_file = zipfile.ZipFile('/content/datasets/ssf0921/p-final-reid/p-final-reid.zip')
for names in zip_file.namelist():
        zip_file.extract(names,'datasets')
zip_file.close()
zip_file = zipfile.ZipFile('/content/datasets/ssfailearning/vc-cloths/vc-cloths.zip')
for names in zip_file.namelist():
        zip_file.extract(names,'datasets')
zip_file.close()

#数据准备

##数据增强

In [5]:
import numpy as np
import pandas as pd
from torch.cuda.amp import GradScaler,autocast
import torch
import math
import random


class RandomErasing(object):
    """ Randomly selects a rectangle region in an image and erases its pixels.
        'Random Erasing Data Augmentation' by Zhong et al.
        See https://arxiv.org/pdf/1708.04896.pdf
    Args:
         probability: The probability that the Random Erasing operation will be performed.
         sl: Minimum proportion of erased area against input image.
         sh: Maximum proportion of erased area against input image.
         r1: Minimum aspect ratio of erased area.
         mean: Erasing value.
    """

    def __init__(self, probability=0.5, sl=0.02, sh=0.4, r1=0.3, mean=(0.4914, 0.4822, 0.4465)):
        self.probability = probability
        self.mean = mean
        self.sl = sl
        self.sh = sh
        self.r1 = r1

    def __call__(self, img):

        if random.uniform(0, 1) >= self.probability:
            return img

        for attempt in range(100):
            area = img.size()[1] * img.size()[2]

            target_area = random.uniform(self.sl, self.sh) * area
            aspect_ratio = random.uniform(self.r1, 1 / self.r1)

            h = int(round(math.sqrt(target_area * aspect_ratio)))
            w = int(round(math.sqrt(target_area / aspect_ratio)))

            if w < img.size()[2] and h < img.size()[1]:
                x1 = random.randint(0, img.size()[1] - h)
                y1 = random.randint(0, img.size()[2] - w)
                if img.size()[0] == 3:
                    img[0, x1:x1 + h, y1:y1 + w] = self.mean[0]
                    img[1, x1:x1 + h, y1:y1 + w] = self.mean[1]
                    img[2, x1:x1 + h, y1:y1 + w] = self.mean[2]
                else:
                    img[0, x1:x1 + h, y1:y1 + w] = self.mean[0]
                return img

        return img

import math
from PIL import Image
import random
import  numpy as np
import random

# This is the code of Random Grayscale Patch Replace

class RGPR(object):

    def __init__(self, probability=0.2, sl=0.02, sh=0.4, r1=0.3):
        self.probability = probability
        self.sl = sl
        self.sh = sh
        self.r1 = r1

    def __call__(self, img):

        new = img.convert("L")   # Convert from here to the corresponding grayscale image
        np_img = np.array(new, dtype=np.uint8)
        img_gray = np.dstack([np_img, np_img, np_img])

        if random.uniform(0, 1) >= self.probability:
            return img

        for attempt in range(100):
            area = img.size[0] * img.size[1]
            target_area = random.uniform(self.sl, self.sh) * area
            aspect_ratio = random.uniform(self.r1, 1 / self.r1)

            h = int(round(math.sqrt(target_area * aspect_ratio)))
            w = int(round(math.sqrt(target_area / aspect_ratio)))

            if w < img.size[1] and h < img.size[0]:
                x1 = random.randint(0, img.size[0] - h)
                y1 = random.randint(0, img.size[1] - w)
                img = np.asarray(img).astype('float')

                img[y1:y1 + h, x1:x1 + w, 0] = img_gray[y1:y1 + h, x1:x1 + w, 0]
                img[y1:y1 + h, x1:x1 + w, 1] = img_gray[y1:y1 + h, x1:x1 + w, 1]
                img[y1:y1 + h, x1:x1 + w, 2] = img_gray[y1:y1 + h, x1:x1 + w, 2]

                img = Image.fromarray(img.astype('uint8'))

                return img

        return img
import torchvision.transforms as T
def build_change_transforms(cfg, training=True):
    normalize_transform = T.Normalize(mean=cfg['mean'], std=cfg['std'])
    if training:
        transform = T.Compose([
            T.Resize(cfg['train_size']),
            T.ColorJitter(),
            T.RandomHorizontalFlip(p=0.5),
            T.Pad(cfg['padding']),
            T.RandomCrop(cfg['train_size']),
            T.RandomGrayscale(p=0.05),
            T.ToTensor(),
            normalize_transform,
            RandomErasing(probability=0.5, mean=cfg['mean']),
            
        ])
    else:
        transform = T.Compose([
            T.Resize(cfg['test_size']),
            T.ToTensor(),
            normalize_transform
        ])

    return transform

def build_origin_transoform(cfg):
  
    transform=T.Compose([
         T.Resize((320,320)),
         T.ToTensor()                
    ])
    return transform

def build_face_transform(cfg,training=True):
  normalize_transform = T.Normalize(mean=cfg['mean'], std=cfg['std'])
  if training:
        transform = T.Compose([
            T.ToPILImage(),
            T.Resize(cfg['train_face_size']),
            T.ColorJitter(),
            T.RandomHorizontalFlip(p=0.5),
            T.Pad(cfg['padding']),
            T.RandomCrop(cfg['train_face_size']),
            T.RandomGrayscale(p=0.05),
            T.RandomAffine(degree=5),
            T.ToTensor(),
            normalize_transform,
            RandomErasing(probability=0.5, mean=cfg['mean'],sh=0.3),
        ])
  else:
        transform = T.Compose([
            T.Resize(cfg['test_face_size']),
            T.ToTensor(),
            normalize_transform
        ])

  return transform
  


##sampler

In [6]:

import copy
import random
import torch
from collections import defaultdict

import numpy as np
from torch.utils.data.sampler import Sampler
class DateRandomIdentitySampler(Sampler):
    """
    Randomly sample N identities, then for each identity,
    randomly sample K instances, therefore batch size is N*K.
    Args:
    - data_source (list): list of (img_path, pid, camid).
    - num_instances (int): number of instances per identity in a batch.
    - batch_size (int): number of examples in a batch.
    """

    def __init__(self, data_source, batch_size, num_instances):
        self.data_source = data_source
        self.batch_size = batch_size
        self.num_instances = num_instances
        self.num_pids_per_batch = self.batch_size // self.num_instances
        self.index_dic = defaultdict(list)
        for index, (_,_, pid, _,_) in enumerate(self.data_source):
            self.index_dic[pid].append(index)
        self.pids = list(self.index_dic.keys())

        # estimate number of examples in an epoch
        self.length = 0
        for pid in self.pids:
            idxs = self.index_dic[pid]
            num = len(idxs)
            if num < self.num_instances:
                num = self.num_instances
            self.length += num - num % self.num_instances

    def __iter__(self):
        batch_idxs_dict = defaultdict(list)

        for pid in self.pids:
            idxs = copy.deepcopy(self.index_dic[pid])
            if len(idxs) < self.num_instances:
                idxs = np.random.choice(idxs, size=self.num_instances, replace=True)
            random.shuffle(idxs)
            batch_idxs = []
            for idx in idxs:
                batch_idxs.append(idx)
                if len(batch_idxs) == self.num_instances:
                    batch_idxs_dict[pid].append(batch_idxs)
                    batch_idxs = []

        avai_pids = copy.deepcopy(self.pids)
        final_idxs = []

        while len(avai_pids) >= self.num_pids_per_batch:
            selected_pids = random.sample(avai_pids, self.num_pids_per_batch)
            for pid in selected_pids:
                batch_idxs = batch_idxs_dict[pid].pop(0)
                final_idxs.extend(batch_idxs)
                if len(batch_idxs_dict[pid]) == 0:
                    avai_pids.remove(pid)

        self.length = len(final_idxs)
        return iter(final_idxs)

    def __len__(self):
        return self.length

##数据集和loader

In [7]:
import glob
import re
import os.path as osp
class Market1501():
    def __init__(self, root='/content/datasets', verbose=True, **kwargs):
        super(Market1501, self).__init__()
        self.dataset_dir = root
        self.train_dir = osp.join(self.dataset_dir, 'bounding_box_train')
        self.query_dir = osp.join(self.dataset_dir, 'query')
        self.gallery_dir = osp.join(self.dataset_dir, 'bounding_box_test')
        self._check_before_run()

        train = self._process_dir(self.train_dir, relabel=True)
        query = self._process_dir(self.query_dir, relabel=False)
        gallery = self._process_dir(self.gallery_dir, relabel=False)

        if verbose:
            print("=> Market1501 loaded")
            self.print_dataset_statistics(train, query, gallery)

        self.train = train
        self.query = query
        self.gallery = gallery

        self.num_train_pids, self.num_train_imgs, self.num_train_cams = self.get_imagedata_info(self.train)
        self.num_query_pids, self.num_query_imgs, self.num_query_cams = self.get_imagedata_info(self.query)
        self.num_gallery_pids, self.num_gallery_imgs, self.num_gallery_cams = self.get_imagedata_info(self.gallery)

    def print_dataset_statistics(self, train, query, gallery):
        num_train_pids, num_train_imgs, num_train_cams = self.get_imagedata_info(train)
        num_query_pids, num_query_imgs, num_query_cams = self.get_imagedata_info(query)
        num_gallery_pids, num_gallery_imgs, num_gallery_cams = self.get_imagedata_info(gallery)

        print("Dataset statistics:")
        print("  ----------------------------------------")
        print("  subset   | # ids | # images | # cameras")
        print("  ----------------------------------------")
        print("  train    | {:5d} | {:8d} | {:9d}".format(num_train_pids, num_train_imgs, num_train_cams))
        print("  query    | {:5d} | {:8d} | {:9d}".format(num_query_pids, num_query_imgs, num_query_cams))
        print("  gallery  | {:5d} | {:8d} | {:9d}".format(num_gallery_pids, num_gallery_imgs, num_gallery_cams))
        print("  ----------------------------------------")
    
    def get_imagedata_info(self, data):
        pids, cams = [], []
        for _, pid, camid,_ in data:
            pids += [pid]
            cams += [camid]
        pids = set(pids)
        cams = set(cams)
        num_pids = len(pids)
        num_cams = len(cams)
        num_imgs = len(data)
        return num_pids, num_imgs, num_cams
    

    def _check_before_run(self):
        """Check if all files are available before going deeper"""
        if not osp.exists(self.dataset_dir):
            raise RuntimeError("'{}' is not available".format(self.dataset_dir))
        if not osp.exists(self.train_dir):
            raise RuntimeError("'{}' is not available".format(self.train_dir))
        if not osp.exists(self.query_dir):
            raise RuntimeError("'{}' is not available".format(self.query_dir))
        if not osp.exists(self.gallery_dir):
            raise RuntimeError("'{}' is not available".format(self.gallery_dir))

    
    ##最终返回的是每张图片的路径，pid，camid
    def _process_dir(self, dir_path, relabel=False):
        img_paths = glob.glob(osp.join(dir_path, '*.jpg'))

        ##匹配文件名中的有用信息，0001_c1代表pid=1，cid=1，
        ##至于为什么是[-\d]+,是因为有-1_c1这种的存在，也应该匹配上
        pattern = re.compile(r'([-\d]+)_c(\d)')

        pid_container = set()
        for img_path in img_paths:
            pid, _ = map(int, pattern.search(img_path).groups())
            if pid == -1: continue  # junk images are just ignored
            pid_container.add(pid)
        pid2label = {pid: label for label, pid in enumerate(pid_container)}

        dataset = []
        for img_path in img_paths:
            pid, camid = map(int, pattern.search(img_path).groups())
            if pid == -1: continue  # junk images are just ignored
            assert 0 <= pid <= 1501  # pid == 0 means background
            assert 1 <= camid <= 6
            camid -= 1  # index starts from 0
            if relabel: pid = pid2label[pid]
            dataset.append((img_path, pid, camid,12))

        return dataset




In [8]:
from fastai.vision import get_image_files
import torch.utils.data as Data
import json


class PDataset:
  def __init__(self,root):
    self.root=root
    self.train_dir=osp.join(self.root,'train')
    self.query_dir=osp.join(self.root,'query')
    self.gallery_dir=osp.join(self.root,'gallery')
    
    train=self._process_dir(self.train_dir)
    query=self._process_dir(self.query_dir)
    gallery=self._process_dir(self.gallery_dir)
    self.train=train
    self.query=query
    self.gallery=gallery
    self.num_train_pids, self.num_train_imgs, self.num_train_cams = self.get_imagedata_info(self.train)
    self.num_query_pids, self.num_query_imgs, self.num_query_cams = self.get_imagedata_info(self.query)
    self.num_gallery_pids, self.num_gallery_imgs, self.num_gallery_cams = self.get_imagedata_info(self.gallery)

  
  def get_imagedata_info(self,data):
    pids,cams=[],[]
    for _,pid,camid,_ in data:
      pids+=[pid]
      cams+=[camid]
    pids=set(pids)
    cams=set(cams)
    num_pids=len(pids)
    num_cams=len(cams)
    num_imgs=len(data)
    return num_pids,num_imgs,num_cams

  def _process_dir(self,dir_path):
    image_names=get_image_files(dir_path)
    dataset=[]
    pid_set=set()
    for name in image_names:
      name=str(name)
      path=name.split('/')[-1]
      pid,_,date,_=path.split('_')
      pid_set.add(pid)
    pid2label={pid: label for label, pid in enumerate(pid_set)}
    for name in image_names:
      name=str(name)
      path=name.split('/')[-1]
      pid,_,date,_=path.split('_')
      pid=pid2label[pid]
      cam=image_cams[path]
      date=date_ids[date]
      dataset.append((name,int(pid),int(cam),int(date)))
    return dataset


In [9]:
import glob
import re
import os.path as osp
class VC_Clothes():
    def __init__(self, root='/content/datasets', verbose=True, **kwargs):
        super(VC_Clothes, self).__init__()
        self.dataset_dir = root
        
     

        train = self._process_dir(self.dataset_dir, relabel=True)
        query = self._process_dir(self.dataset_dir, relabel=False)
        gallery = self._process_dir(self.dataset_dir, relabel=False)

        if verbose:
            print("=> VC loaded")
            self.print_dataset_statistics(train, query, gallery)

        self.train = train
        self.query = query
        self.gallery = gallery

        self.num_train_pids, self.num_train_imgs, self.num_train_cams = self.get_imagedata_info(self.train)
        self.num_query_pids, self.num_query_imgs, self.num_query_cams = self.get_imagedata_info(self.query)
        self.num_gallery_pids, self.num_gallery_imgs, self.num_gallery_cams = self.get_imagedata_info(self.gallery)


    def print_dataset_statistics(self, train, query, gallery):
        num_train_pids, num_train_imgs, num_train_cams = self.get_imagedata_info(train)
        num_query_pids, num_query_imgs, num_query_cams = self.get_imagedata_info(query)
        num_gallery_pids, num_gallery_imgs, num_gallery_cams = self.get_imagedata_info(gallery)

        print("Dataset statistics:")
        print("  ----------------------------------------")
        print("  subset   | # ids | # images | # cameras")
        print("  ----------------------------------------")
        print("  train    | {:5d} | {:8d} | {:9d}".format(num_train_pids, num_train_imgs, num_train_cams))
        print("  query    | {:5d} | {:8d} | {:9d}".format(num_query_pids, num_query_imgs, num_query_cams))
        print("  gallery  | {:5d} | {:8d} | {:9d}".format(num_gallery_pids, num_gallery_imgs, num_gallery_cams))
        print("  ----------------------------------------")
    
    def get_imagedata_info(self, data):
        pids, cams = [], []
        for _, pid, camid,_ in data:
            pids += [pid]
            cams += [camid]
        pids = set(pids)
        cams = set(cams)
        num_pids = len(pids)
        num_cams = len(cams)
        num_imgs = len(data)
        return num_pids, num_imgs, num_cams
    

  
    
    ##最终返回的是每张图片的路径，pid，camid
    def _process_dir(self, dir_path, relabel=False):
        image_names=get_image_files(dir_path)
        
        pid_container = set()
        for name in image_names:
            name=str(name)
            pid,date,_,_=name.split('-')
            if pid == -1: continue  # junk images are just ignored
            pid_container.add(pid)
        pid2label = {pid: label for label, pid in enumerate(pid_container)}

        dataset = []
        for name in image_names:
            name=str(name)
            pid,date,_,_=name.split('-')
            if relabel: pid = pid2label[pid]
            dataset.append((name, pid, 1,1))
        return dataset



In [10]:
class CUHK03():
    def __init__(self, root='/content/datasets', verbose=True, **kwargs):
        super(CUHK03, self).__init__()
        self.dataset_dir = root
        
     

        train = self._process_dir(self.dataset_dir, relabel=True)
        query = self._process_dir(self.dataset_dir, relabel=False)
        gallery = self._process_dir(self.dataset_dir, relabel=False)

        if verbose:
            print("=> cuhk loaded")
            self.print_dataset_statistics(train, query, gallery)

        self.train = train
        self.query = query
        self.gallery = gallery

        self.num_train_pids, self.num_train_imgs, self.num_train_cams = self.get_imagedata_info(self.train)
        self.num_query_pids, self.num_query_imgs, self.num_query_cams = self.get_imagedata_info(self.query)
        self.num_gallery_pids, self.num_gallery_imgs, self.num_gallery_cams = self.get_imagedata_info(self.gallery)


    def print_dataset_statistics(self, train, query, gallery):
        num_train_pids, num_train_imgs, num_train_cams = self.get_imagedata_info(train)
        num_query_pids, num_query_imgs, num_query_cams = self.get_imagedata_info(query)
        num_gallery_pids, num_gallery_imgs, num_gallery_cams = self.get_imagedata_info(gallery)

        print("Dataset statistics:")
        print("  ----------------------------------------")
        print("  subset   | # ids | # images | # cameras")
        print("  ----------------------------------------")
        print("  train    | {:5d} | {:8d} | {:9d}".format(num_train_pids, num_train_imgs, num_train_cams))
        print("  query    | {:5d} | {:8d} | {:9d}".format(num_query_pids, num_query_imgs, num_query_cams))
        print("  gallery  | {:5d} | {:8d} | {:9d}".format(num_gallery_pids, num_gallery_imgs, num_gallery_cams))
        print("  ----------------------------------------")
    
    def get_imagedata_info(self, data):
        pids, cams = [], []
        for _, pid, camid,_ in data:
            pids += [pid]
            cams += [camid]
        pids = set(pids)
        cams = set(cams)
        num_pids = len(pids)
        num_cams = len(cams)
        num_imgs = len(data)
        return num_pids, num_imgs, num_cams
    

  
    
    ##最终返回的是每张图片的路径，pid，camid
    def _process_dir(self, dir_path, relabel=False):
        image_names=get_image_files(dir_path)
        
        pid_container = set()
        for name in image_names:
            name=str(name)
            path=name.split('/')[-1]
            pid,date,_,_=path.split('_')
            if pid == -1: continue  # junk images are just ignored
            pid_container.add(pid)
        pid2label = {pid: label for label, pid in enumerate(pid_container)}

        dataset = []
        for name in image_names:
            name=str(name)
            path=name.split('/')[-1]
            pid,date,_,_=path.split('_')
            if relabel: pid = pid2label[pid]
            dataset.append((name, pid, 1,1))
        return dataset

In [11]:
class Real28():
    def __init__(self, root='/content/datasets', verbose=True, **kwargs):
        super(Real28, self).__init__()
        self.dataset_dir = root
        
     

        train = self._process_dir(self.dataset_dir, relabel=True)
        query = self._process_dir(self.dataset_dir, relabel=False)
        gallery = self._process_dir(self.dataset_dir, relabel=False)

        if verbose:
            print("=> real loaded")
            self.print_dataset_statistics(train, query, gallery)

        self.train = train
        self.query = query
        self.gallery = gallery

        self.num_train_pids, self.num_train_imgs, self.num_train_cams = self.get_imagedata_info(self.train)
        self.num_query_pids, self.num_query_imgs, self.num_query_cams = self.get_imagedata_info(self.query)
        self.num_gallery_pids, self.num_gallery_imgs, self.num_gallery_cams = self.get_imagedata_info(self.gallery)


    def print_dataset_statistics(self, train, query, gallery):
        num_train_pids, num_train_imgs, num_train_cams = self.get_imagedata_info(train)
        num_query_pids, num_query_imgs, num_query_cams = self.get_imagedata_info(query)
        num_gallery_pids, num_gallery_imgs, num_gallery_cams = self.get_imagedata_info(gallery)

        print("Dataset statistics:")
        print("  ----------------------------------------")
        print("  subset   | # ids | # images | # cameras")
        print("  ----------------------------------------")
        print("  train    | {:5d} | {:8d} | {:9d}".format(num_train_pids, num_train_imgs, num_train_cams))
        print("  query    | {:5d} | {:8d} | {:9d}".format(num_query_pids, num_query_imgs, num_query_cams))
        print("  gallery  | {:5d} | {:8d} | {:9d}".format(num_gallery_pids, num_gallery_imgs, num_gallery_cams))
        print("  ----------------------------------------")
    
    def get_imagedata_info(self, data):
        pids, cams = [], []
        for _, pid, camid,_ in data:
            pids += [pid]
            cams += [camid]
        pids = set(pids)
        cams = set(cams)
        num_pids = len(pids)
        num_cams = len(cams)
        num_imgs = len(data)
        return num_pids, num_imgs, num_cams
    
    
    ##最终返回的是每张图片的路径，pid，camid
    def _process_dir(self, dir_path, relabel=False):
        image_names=get_image_files(dir_path)
        pid_container = set()
        for name in image_names:
            name=str(name)
            path=name.split('/')[-1]
            pid,date,_,_=path.split('_')
            if pid == -1: continue  # junk images are just ignored
            pid_container.add(pid)
        pid2label = {pid: label for label, pid in enumerate(pid_container)}
        dataset = []
        for name in image_names:
            name=str(name)
            path=name.split('/')[-1]
            pid,date,_,_=path.split('_')
            if relabel: pid = pid2label[pid]
            dataset.append((name, pid, 1,1))
        return dataset

In [12]:



import torch.utils.data as Data
from PIL import Image


class ImageDataset(Data.Dataset):
    def __init__(self, dataset, transform=None,ori_transform=None,pid_add=0):
        self.dataset = dataset
        self.transform = transform
        self.ori_transform=ori_transform
        self.pid_add=pid_add

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        img_path, pid, camid,date = self.dataset[index]
        img = Image.open(img_path)
        if self.transform is not None:
            img = self.transform(img)
        if self.ori_transform is not None:
            ori_img=self.transform(img)
        return img,ori_img ,pid+self.pid_add, camid, img_path,date
def train_collate_fn1(batch):
    imgs, ori_img,pids, _, _,_ = zip(*batch)
    pids = torch.tensor(pids, dtype=torch.int64)
    return torch.stack(imgs, dim=0),torch.stack(ori_img,dim=0), pids

###对于验证集而言，为了提高验证的真实性，我们应该防止同一摄像头的图片进入验证（同一摄像头相当于数据泄露）
def val_collate_fn1(batch):
    imgs, ori_img,pids, camids, _,dates = zip(*batch)
    return torch.stack(imgs, dim=0), torch.stack(ori_img,dim=0),pids, camids,dates



def make_dataloader(datasets,cfg,pid_add=0):
    train_transform=build_change_transforms(cfg,training=True)
    test_transform=build_change_transforms(cfg,training=False)
    ori_transform=build_origin_transform(cfg)
   
    data=datasets

    num_classes=data.num_train_pids
    train_set=ImageDataset(data.train,train_transform,ori_transform,pid_add=pid_add)

    if cfg['SAMPLER']=='softmax':
        train_loader=Data.DataLoader(train_set,batch_size=cfg['train_bs'],
                                     shuffle=True,collate_fn=train_collate_fn1)
    else:
        train_loader=Data.DataLoader(train_set,batch_size=cfg['train_bs'],
                                     sampler=DateRandomIdentitySampler(data.train,
                                    cfg['train_bs'],cfg['train_K_instances']),
                                    num_workers=cfg['num_workers'],
                                    collate_fn=train_collate_fn1)
    
    val_set = ImageDataset(data.query + data.gallery, test_transform,ori_transform)
    val_loader = Data.DataLoader(
        val_set, batch_size=cfg['test_bs'], shuffle=False, num_workers=cfg['num_workers'],
        collate_fn=val_collate_fn1
    )
    return train_loader, val_loader, len(data.query), num_classes

def MAKE_DATALOADER(cfg):
  train_names=cfg['train_data']
  val_name=cfg['val_data']
  val_loader_dict={}
  train_loaders=[]
  all_num_class=0
  for name in train_names:
    dataset=data_rott[name]
    train_loader,val_loader,num_query,num_classes=make_dataloader(dataset,cfg,all_num_class)
    train_loaders.append(train_loader)
    val_loader_dict[name]=(val_loader,num_query)
    all_num_class+=num_classes
  val_loader,num_query=val_loader_dict[val_name]
  return train_loaders,val_loader,num_query,all_num_class
 

#loss

In [13]:
import torch.nn as nn
class CrossEntropyLabelSmoothLoss(nn.Module):
    """Cross entropy loss with label smoothing regularizer.
    Reference:
    Szegedy et al. Rethinking the Inception Architecture for Computer Vision. CVPR 2016.
    Equation: y = (1 - epsilon) * y + epsilon / K.
    Args:
        num_classes (int): number of classes.
        epsilon (float): weight.
    """

    def __init__(self, num_classes, epsilon=0.1, use_gpu=True):
        super(CrossEntropyLabelSmoothLoss, self).__init__()
        self.num_classes = num_classes
        self.epsilon = epsilon
        self.use_gpu = use_gpu
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def forward(self, inputs, targets):
        """
        Args:
            inputs: prediction matrix (before softmax) with shape (batch_size, num_classes)
            targets: ground truth labels with shape (num_classes)
        """
        
        log_probs = self.logsoftmax(inputs)
        targets = torch.zeros(log_probs.size()).scatter_(1, targets.unsqueeze(1).cpu(), 1)
        if self.use_gpu: targets = targets.cuda()
        targets = (1 - self.epsilon) * targets + self.epsilon / self.num_classes
        loss = (- targets * log_probs).mean(0).sum()
        return loss


from typing import Tuple
from torch import Tensor
def convert_label_to_similarity(normed_feature: Tensor, label: Tensor) -> Tuple[Tensor, Tensor]:
    similarity_matrix = normed_feature @ normed_feature.transpose(1, 0)
    label_matrix = label.unsqueeze(1) == label.unsqueeze(0)

    positive_matrix = label_matrix.triu(diagonal=1)
    negative_matrix = label_matrix.logical_not().triu(diagonal=1)

    similarity_matrix = similarity_matrix.view(-1)
    positive_matrix = positive_matrix.view(-1)
    negative_matrix = negative_matrix.view(-1)
    return similarity_matrix[positive_matrix], similarity_matrix[negative_matrix]


def normalize(x, axis=-1):

	x = 1. * x / (torch.norm(x, 2, axis, keepdim=True).expand_as(x) + 1e-12)
	return x
class CircleLoss(nn.Module):
    def __init__(self, m: float, gamma: float) -> None:
        super(CircleLoss, self).__init__()
        self.m = m
        self.gamma = gamma
        self.soft_plus = nn.Softplus()


    def forward(self,feat,label) -> Tensor:

        feat=normalize(feat,axis=-1)
        sp, sn = convert_label_to_similarity(feat, label)
        ap = torch.clamp_min(- sp.detach() + 1 + self.m, min=0.)
        an = torch.clamp_min(sn.detach() + self.m, min=0.)

        delta_p = 1 - self.m
        delta_n = self.m

        logit_p = - ap * (sp - delta_p) * self.gamma
        logit_n = an * (sn - delta_n) * self.gamma

        loss = self.soft_plus(torch.logsumexp(logit_n, dim=0) + torch.logsumexp(logit_p, dim=0))

        return loss

import torch
from torch.autograd import Variable


def hard_sigmoid(x, c=2.5):
    if x < -c:
        return 0
    if x > c:
        return 1
    return x / (2 * c) + 0.5

class Adaptive_l2_loss(nn.Module):
  def __init__(self,kernel_regularization_factor=0.005,bias_regularization_factor=0.005):
    self.kernel_regularization_factor=kernel_regularization_factor
    self.bias_regularization_factor=bias_regularization_factor
    super(Adaptive_l2_loss,self).__init__()

  def forward(self,out,pid):
    model=out['model']
    l2_reg = Variable(torch.tensor(0.0), requires_grad=True)
    i = 0
    for name, params in model.named_parameters():
        if "conv" in name:
            if "weight" in name:
                l2_reg = l2_reg + hard_sigmoid(self.kernel_regularization_factor * model.adaptive_param[i] * params.norm(2))
                i += 1
            if "bias" in name:
                l2_reg = l2_reg + hard_sigmoid(self.bias_regularization_factor * model.adaptive_param[i] * params.norm(2))
                i += 1

    return l2_reg

#scheduler

In [14]:
from bisect import bisect_right
import torch




class WarmupMultiStepLR(torch.optim.lr_scheduler._LRScheduler):
    def __init__(
        self,
        optimizer,
        milestones,
        gamma=0.1,
        warmup_factor=1.0 / 3,
        warmup_iters=500,
        warmup_method="linear",
        last_epoch=-1,
    ):
        if not list(milestones) == sorted(milestones):
            raise ValueError(
                "Milestones should be a list of" " increasing integers. Got {}",
                milestones,
            )

        if warmup_method not in ("constant", "linear"):
            raise ValueError(
                "Only 'constant' or 'linear' warmup_method accepted"
                "got {}".format(warmup_method)
            )
        self.milestones = milestones
        self.gamma = gamma
        self.warmup_factor = warmup_factor
        self.warmup_iters = warmup_iters
        self.warmup_method = warmup_method
        super(WarmupMultiStepLR, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        warmup_factor = 1
        if self.last_epoch < self.warmup_iters:
            if self.warmup_method == "constant":
                warmup_factor = self.warmup_factor
            elif self.warmup_method == "linear":
                alpha = self.last_epoch / self.warmup_iters
                warmup_factor = self.warmup_factor * (1 - alpha) + alpha
        return [
            base_lr
            * warmup_factor
            * self.gamma ** bisect_right(self.milestones, self.last_epoch)
            for base_lr in self.base_lrs
        ]            
class Flat(torch.optim.lr_scheduler._LRScheduler):
  def __init__(self,optimizer,cfg,anneal_start=0.65,last_epoch=-1):
    self.epochs=cfg['epochs']
    self.start=anneal_start
    super(Flat,self).__init__(optimizer,last_epoch)
  
  def get_lr(self):
    if self.last_epoch<self.epochs*self.start:
      return [base_lr for base_lr in self.base_lrs]
    else:
      return [
        base_lr*(1+math.cos(math.pi*self.last_epoch/5))/2  
        for base_lr in self.base_lrs    
      ]
     


#辅助函数

In [15]:
class Checkpoint:
  def __init__(self,ckpt):
    self.ckpt=ckpt
    self.init=-1000
  
  def __call__(self,metric,model,optimizer):
    if metric>self.init:
      self.init=metric
      torch.save({'model':model.state_dict(),'optimizer':optimizer.state_dict()},self.ckpt)
    
from thop import profile
from thop import clever_format

def count_your_model(model, x, y):
  outs=model(x)
  
def get_pf(model):


  device=torch.device('cuda:0')
  input = torch.randn(1, 3, 384, 128)
  input=input.to(device)
  flops, params = profile(model, inputs=(input, ))
  flops, params = clever_format([flops, params], "%.3f")
  return flops,params

class EMA():
    def __init__(self, model, decay):
        self.model = model
        self.decay = decay
        self.shadow = {}
        self.backup = {}

    def register(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                self.shadow[name] = param.data.clone()

    def update(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                new_average = (1.0 - self.decay) * param.data + self.decay * self.shadow[name]
                self.shadow[name] = new_average.clone()
    
    def apply_shadow(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.shadow
                self.backup[name] = param.data
                param.data = self.shadow[name]
    
    def restore(self):
        for name, param in self.model.named_parameters():
            if param.requires_grad:
                assert name in self.backup
                param.data = self.backup[name]
        self.backup = {}

#Metric

In [16]:
'''reid计算map函数'''
def MARKET_EVAL_FUNC(dismat,q_pids,g_pids,q_camids,g_camids,max_rank=50):
    '''
    :param dismat: 每个query与其对应查询结果的距离
    :param q_pids: 每个query行人的id
    :param g_pids: 每个查询结果的行人的id
    :param q_camids: 摄像头id，对于同一个query，如果查询结果与query都来自一个摄像头，则丢弃
    :param g_camids: 同上
    :param max_rank:
    :return:
    '''
    num_q,num_g=dismat.shape
    if num_g<max_rank:
        max_rank=num_g

    ##根据相似度进行排序
    indices=np.argsort(dismat,axis=1)
    
    
    ##在排序结果找到同id图片，用于计算cmc
    matchs=(g_pids[indices]==q_pids[:,np.newaxis]).astype(np.int32)
    


    all_cmc=[]
    all_AP=[]
    num_valid_q=0

    for q_idx in range(num_q):
        q_pid=q_pids[q_idx]
        q_camid=q_camids[q_idx]

        '''当前query对应查询结果的排序'''
        order=indices[q_idx]

        '''若查询结果与query同摄像机且同id，删除'''
        remove=(g_pids[order]==q_pid)&(g_camids[order]==q_camid)
        keep=np.invert(remove)

        '''计算CMC'''

        '''首先先筛选'''
        orig_cmc=matchs[q_idx][keep]

        '''全是false'''
        if not np.any(orig_cmc):
            continue

        

        '''累加和，因为orig_cmc里面是true，false，true为1，得到cmc列表'''
        cmc=orig_cmc.cumsum()
        cmc[cmc>1]=1
        all_cmc.append(cmc[:max_rank])
        num_valid_q+=1

        '''计算map'''

        '''预测正确的数量'''
        num_rel=orig_cmc.sum()

        '''累加和，用于计算'''
        tmp_cmc=orig_cmc.cumsum()

        '''计算准确率，前k个查询结果的准确率'''
        tmp_cmc=[x/(i+1) for i,x in enumerate(tmp_cmc)]

        '''计算召回率，召回率每次变化都是当查询结果id等于query时，故乘orig_cmc'''
        '''最终的结果类似 0,0,1,0,0,2,0,0,3这种，每个有值的都是查询正确的'''
        '''然后后面除以查询结果中同id数量，也就是上面num_rel就是找召回率'''
        '''但是这里其实已经提前乘上准确率了，最终结果类似于 0,0,1/3,0,0,2/6,0,0,3/9'''
        tmp_cmc=np.asarray(tmp_cmc)*orig_cmc

        '''以上为例，除以同id数量后，假设为五个'''
        '''0，0，1/3 /5，0，0，2/6 /5，0，0，3/9 /5'''
        '''等价于 0,0, 1/3(准确率)*1/5(召回率) ....'''
        
        if num_rel==0:
          AP=0
          print('AP=0')
        else:  
          AP = tmp_cmc.sum() / num_rel
          
        all_AP.append(AP)

    all_cmc = np.asarray(all_cmc).astype(np.float32)
    all_cmc = all_cmc.sum(0) / num_valid_q
    if len(all_AP)==0:
      print('all_ap=0')
      return all_cmc,0
    mAP = np.mean(all_AP)

    return all_cmc, mAP


class MARKET_MAP():
    def __init__(self,num_query,max_rank=50,feat_norm=True,one_day=True,all=False,date_length=11):
        '''

        :param num_query: 查询数量
        :param max_rank: 每个query限定的最大查询结果数量
        :param feat_norm:是否对特征进行l2norm
        '''
        self.num_query=num_query
        self.max_rank=max_rank
        self.feat_norm=feat_norm
        self.one_day=one_day
        self.all=all
        self.date_length=date_length
        self.reset()

    def reset(self):
        '''
        feats:模型返回的特征
        pids: 图片中行人的id
        camids:图片摄像头的id
        :return:
        '''
        self.feats=[]
        self.pids=[]
        self.camids=[]
        self.dates=[]

    def update(self,output):
        feat,pid,camid,date=output
    
        self.feats.append(feat)
        self.pids.append(pid)
        self.camids.append(camid)
        self.dates.append(date)

    def compute(self):
        feats=torch.stack(self.feats,dim=0)
          
        if self.feat_norm:
            feats=torch.nn.functional.normalize(feats,p=2)

        
        '''用于查询的图片的特征及其id和摄像头id'''
        
        qf=feats[:self.num_query]
        q_pids=np.asarray(self.pids[:self.num_query])
        q_camids = np.asarray(self.camids[:self.num_query])
        q_dates=np.asarray(self.dates[:self.num_query])  

        '''每个query查询结果的特征及其id和摄像头id'''
        gf = feats[self.num_query:]
        g_pids = np.asarray(self.pids[self.num_query:])
        g_camids = np.asarray(self.camids[self.num_query:])
        g_dates=np.asarray(self.dates[self.num_query:])
        if self.all:
          m,n=qf.shape[0],gf.shape[0]
      
          '''计算qf每个query和对应查询结果的距离'''
          distmat = torch.pow(qf, 2).sum(dim=1, keepdim=True).expand(m, n) + \
                    torch.pow(gf, 2).sum(dim=1, keepdim=True).expand(n, m).t()
        
          distmat.addmm_(1, -2, qf, gf.t())
          distmat = distmat.cpu().numpy()
        
          cmc,mAP=MARKET_EVAL_FUNC(distmat,q_pids,g_pids,q_camids,g_camids)
          return mAP
        else:
          if self.one_day:
            
            mean_map=0
            count=0
            for date in range(self.date_length):
              date=date+1
              q_index=np.where(q_dates==date)
              
              date_qf=qf[q_index]
              date_q_pids=q_pids[q_index]
              date_q_camids=q_camids[q_index]

              g_index=np.where(g_dates==date)
              date_gf=gf[g_index]
              date_g_pids=g_pids[g_index]
              date_g_camids=g_camids[g_index]
              m,n=date_qf.shape[0],date_gf.shape[0]

              if m==0:
                continue
              distmat = torch.pow(date_qf, 2).sum(dim=1, keepdim=True).expand(m, n) + \
                    torch.pow(date_gf, 2).sum(dim=1, keepdim=True).expand(n, m).t()
              
              distmat.addmm_(1, -2, date_qf, date_gf.t())
              distmat = distmat.cpu().numpy()
              
              cmc,mAP=MARKET_EVAL_FUNC(distmat,date_q_pids,date_g_pids,date_q_camids,date_g_camids)
              mean_map+=mAP
              count+=1
            
            if count==0:
              return 0
            return mean_map/count
          else:
           
            mean_map=0
            count=0
            for date in range(self.date_length):
              date=date+1
              q_index=np.where(q_dates==date)
              
              date_qf=qf[q_index]
              date_q_pids=q_pids[q_index]
              date_q_camids=q_camids[q_index]

              g_index=np.where(g_dates!=date)
              date_gf=gf[g_index]
              date_g_pids=g_pids[g_index]
              date_g_camids=g_camids[g_index]
              m,n=date_qf.shape[0],date_gf.shape[0]

              if m==0 or n==0:
                continue
              distmat = torch.pow(date_qf, 2).sum(dim=1, keepdim=True).expand(m, n) + \
                    torch.pow(date_gf, 2).sum(dim=1, keepdim=True).expand(n, m).t()
              
              distmat.addmm_(1, -2, date_qf, date_gf.t())
              distmat = distmat.cpu().numpy()
              
              cmc,mAP=MARKET_EVAL_FUNC(distmat,date_q_pids,date_g_pids,date_q_camids,date_g_camids)
              mean_map+=mAP
              count+=1
            if count==0:
              return 0
            return mean_map/count

#model

##非主要函数

In [63]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from lambda_networks import LambdaLayer
def weights_init_kaiming(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        nn.init.kaiming_normal_(m.weight, a=0, mode='fan_out')
        nn.init.constant_(m.bias, 0.0)
    elif classname.find('Conv') != -1:
        nn.init.kaiming_normal_(m.weight, a=0, mode='fan_in')
        if m.bias is not None:
            nn.init.constant_(m.bias, 0.0)
    elif classname.find('BatchNorm') != -1:
        if m.affine:
            nn.init.constant_(m.weight, 1.0)
            nn.init.constant_(m.bias, 0.0)


def weights_init_classifier(m):
    classname = m.__class__.__name__
    if classname.find('Linear') != -1:
        nn.init.normal_(m.weight, std=0.001)
        if m.bias:
            nn.init.constant_(m.bias, 0.0)

class ConvBlock(nn.Module):
    """Basic convolutional block.

    convolution + batch normalization + relu.
    Args:
        in_c (int): number of input channels.
        out_c (int): number of output channels.
        k (int or tuple): kernel size.
        s (int or tuple): stride.
        p (int or tuple): padding.
    """

    def __init__(self, in_c, out_c, k, s=1, p=0):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_c, out_c, k, stride=s, padding=p)
        self.bn = nn.BatchNorm2d(out_c)

    def forward(self, x):
        return F.relu(self.bn(self.conv(x)))


class InceptionA(nn.Module):

    def __init__(self, in_channels, out_channels):
        super(InceptionA, self).__init__()
        mid_channels = out_channels // 4

        self.stream1 = nn.Sequential(
            ConvBlock(in_channels, mid_channels, 1),
            ConvBlock(mid_channels, mid_channels, 3, p=1),
        )
        self.stream2 = nn.Sequential(
            ConvBlock(in_channels, mid_channels, 1),
            ConvBlock(mid_channels, mid_channels, 3, p=1),
        )
        self.stream3 = nn.Sequential(
            ConvBlock(in_channels, mid_channels, 1),
            ConvBlock(mid_channels, mid_channels, 3, p=1),
        )
        self.stream4 = nn.Sequential(
            nn.AvgPool2d(3, stride=1, padding=1),
            ConvBlock(in_channels, mid_channels, 1),
        )

    def forward(self, x):
        s1 = self.stream1(x)
        s2 = self.stream2(x)
        s3 = self.stream3(x)
        s4 = self.stream4(x)
        y = torch.cat([s1, s2, s3, s4], dim=1)
        return y


class InceptionB(nn.Module):

    def __init__(self, in_channels, out_channels):
        super(InceptionB, self).__init__()
        mid_channels = out_channels // 4

        self.stream1 = nn.Sequential(
            ConvBlock(in_channels, mid_channels, 1),
            ConvBlock(mid_channels, mid_channels, 3, s=2, p=1),
        )
        self.stream2 = nn.Sequential(
            ConvBlock(in_channels, mid_channels, 1),
            ConvBlock(mid_channels, mid_channels, 3, p=1),
            ConvBlock(mid_channels, mid_channels, 3, s=2, p=1),
        )
        self.stream3 = nn.Sequential(
            nn.MaxPool2d(3, stride=2, padding=1),
            ConvBlock(in_channels, mid_channels * 2, 1),
        )

    def forward(self, x):
        s1 = self.stream1(x)
        s2 = self.stream2(x)
        s3 = self.stream3(x)
        y = torch.cat([s1, s2, s3], dim=1)
        return y

class Inception_block(nn.Module):
    def __init__(self,channel1,channel2):
        self.block1=InceptionA(channel1,channel2)
        self.block2=InceptionB(channel2,channel2)
        super(Inception_block, self).__init__()
    def forward(self,x):
        x=self.block1(x)
        x=self.block2(x)
        return x


class DW_conv(nn.Module):
    def __init__(self, in_ch, out_ch,stride=1):
        super(DW_conv, self).__init__()
        self.depth_conv = nn.Conv2d(
            in_channels=in_ch,
            out_channels=in_ch,
            kernel_size=3,
            stride=1,
            padding=1,
            groups=in_ch
        )
        self.point_conv = nn.Conv2d(
            in_channels=in_ch,
            out_channels=out_ch,
            kernel_size=1,
            stride=stride,
            padding=0,
            groups=1
        )
        self.bn=nn.BatchNorm2d(out_ch)

    def forward(self, input):
        out = self.depth_conv(input)
        out = self.point_conv(out)
        out=self.bn(out)
        return out

class ShuffleBlock(nn.Module):
    def __init__(self, groups):
        super(ShuffleBlock, self).__init__()
        self.groups = groups

    def forward(self, x):
        '''Channel shuffle: [N,C,H,W] -> [N,g,C/g,H,W] -> [N,C/g,g,H,w] -> [N,C,H,W]'''
        N,C,H,W = x.size()
        g = self.groups
        return x.view(N,g,C//g,H,W).permute(0,2,1,3,4).reshape(N,C,H,W)

class Shuffle_A(nn.Module):
    def __init__(self,inchannel):
        split_channel=int(inchannel//2)
        self.split_channel=split_channel
        self.conv1=nn.Conv2d(split_channel,inchannel,kernel_size=1)
        self.dw_conv=DW_conv(inchannel,inchannel)
        self.conv2=nn.Conv2d(inchannel,split_channel,kernel_size=1)
        self.bn_relu1=self.make_bn_relu(inchannel)
        self.bn_relu2=self.make_bn_relu(inchannel)
        self.relu=nn.ReLU()
        self.channel_shuffle=ShuffleBlock(4)
        super(Shuffle_A, self).__init__()

    def make_bn_relu(self,in_channel):
        return nn.Sequential(
            nn.BatchNorm2d(in_channel),
            nn.ReLU()
        )
    def forward(self,x):
        x_a=x[:,:self.split_channel,:,:]
        x_b=x[:,self.split_channel:,:,:]

        x_b=self.conv1(x_b)
        x_b=self.bn_relu1(x_b)
        x_b=self.dw_conv(x_b)
        x_b=self.conv2(x_b)
        x_b=self.bn_relu2(x_b)
        
        x=torch.cat([x_a,x_b],dim=1)
        x=self.channel_shuffle(x)


class Shuffle_B(nn.Module):
    def __init__(self, inchannel,out_channel,stride=1):
        split_channel = int(inchannel // 2)
        self.split_channel = split_channel
        self.conv1 = nn.Conv2d(split_channel, inchannel,kernel_size=1)
        self.dw_conv = DW_conv(inchannel,inchannel,stride)
        self.conv2 = nn.Conv2d(inchannel, out_channel,kernel_size=1)
        self.bn_relu1 = self.make_bn_relu(inchannel)
        self.bn_relu2 = self.make_bn_relu(out_channel)

        self.dw_conv2=DW_conv(split_channel, split_channel,stride)
        self.conv3=nn.Conv2d(split_channel,out_channel,kernel_size=1)
        self.bn_relu3=self.make_bn_relu(out_channel)

        self.channel_shuffle = ShuffleBlock(4)
        super(Shuffle_B, self).__init__()
    def make_bn_relu(self, in_channel):
        return nn.Sequential(
            nn.BatchNorm2d(in_channel),
            nn.ReLU()
        )

    def forward(self, x):
        x_a = x[:, :self.split_channel, :, :]
        x_b = x[:, self.split_channel:, :, :]

        x_b = self.conv1(x_b)
        x_b = self.bn_relu1(x_b)
        x_b = self.dw_conv(x_b)
        x_b = self.conv2(x_b)
        x_b = self.bn_relu2(x_b)

        x_a=self.dw_conv2(x_a)
        x_a=self.conv3(x_a)
        x_a=self.bn_relu3(x_a)
        x = torch.cat([x_a, x_b], dim=1)
        x = self.channel_shuffle(x)



class SpatialAttn(nn.Module):
    """Spatial Attention (Sec. 3.1.I.1)"""

    def __init__(self):
        super(SpatialAttn, self).__init__()
        self.conv1 = ConvBlock(1, 1, 3, s=2, p=1)
        self.conv2 = ConvBlock(1, 1, 1)

    def forward(self, x):
        # global cross-channel averaging
        x = x.mean(1, keepdim=True)
        # 3-by-3 conv
        x = self.conv1(x)
        # bilinear resizing
        x = F.upsample(
            x, (x.size(2) * 2, x.size(3) * 2),
            mode='bilinear',
            align_corners=True
        )
        # scaling conv
        x = self.conv2(x)
        return x


class ChannelAttn(nn.Module):
    """Channel Attention (Sec. 3.1.I.2)"""

    def __init__(self, in_channels, reduction_rate=16):
        super(ChannelAttn, self).__init__()
        assert in_channels % reduction_rate == 0
        self.conv1 = ConvBlock(in_channels, in_channels // reduction_rate, 1)
        self.conv2 = ConvBlock(in_channels // reduction_rate, in_channels, 1)

    def forward(self, x):
        # squeeze operation (global average pooling)
        x = F.avg_pool2d(x, x.size()[2:])
        # excitation operation (2 conv layers)
        x = self.conv1(x)
        x = self.conv2(x)
        return x


class SoftAttn(nn.Module):
    """Soft Attention (Sec. 3.1.I)

    Aim: Spatial Attention + Channel Attention

    Output: attention maps with shape identical to input.
    """

    def __init__(self, in_channels):
        super(SoftAttn, self).__init__()
        self.spatial_attn = SpatialAttn()
        self.channel_attn = ChannelAttn(in_channels)
        self.conv = ConvBlock(in_channels, in_channels, 1)

    def forward(self, x):
        y_spatial = self.spatial_attn(x)
        y_channel = self.channel_attn(x)
        y = y_spatial * y_channel
        y = torch.sigmoid(self.conv(y))
        return y


class HardAttn(nn.Module):
    """Hard Attention (Sec. 3.1.II)"""

    def __init__(self, in_channels):
        super(HardAttn, self).__init__()
        self.fc = nn.Linear(in_channels, 4 * 2)
        self.init_params()

    def init_params(self):
        self.fc.weight.data.zero_()
        self.fc.bias.data.copy_(
            torch.tensor(
                [0, -0.75, 0, -0.25, 0, 0.25, 0, 0.75], dtype=torch.float
            )
        )

    def forward(self, x):
        # squeeze operation (global average pooling)
        x = F.avg_pool2d(x, x.size()[2:]).view(x.size(0), x.size(1))
        # predict transformation parameters
        theta = torch.tanh(self.fc(x))
        theta = theta.view(-1, 4, 2)
        return theta


class HarmAttn(nn.Module):
    """Harmonious Attention (Sec. 3.1)"""

    def __init__(self, in_channels):
        super(HarmAttn, self).__init__()
        self.soft_attn = SoftAttn(in_channels)
        self.hard_attn = HardAttn(in_channels)

    def forward(self, x):
        y_soft_attn = self.soft_attn(x)
        theta = self.hard_attn(x)
        return y_soft_attn, theta


def _make_divisible(v, divisor, min_value=None):
    """
    This function is taken from the original tf repo.
    It ensures that all layers have a channel number that is divisible by 8
    It can be seen here:
    https://github.com/tensorflow/models/blob/master/research/slim/nets/mobilenet/mobilenet.py
    """
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v


def hard_sigmoid(x, inplace: bool = False):
    if inplace:
        return x.add_(3.).clamp_(0., 6.).div_(6.)
    else:
        return F.relu6(x + 3.) / 6.

import math
class SqueezeExcite(nn.Module):
    def __init__(self, in_chs, se_ratio=0.25, reduced_base_chs=None,
                 act_layer=nn.ReLU, gate_fn=hard_sigmoid, divisor=4, **_):
        super(SqueezeExcite, self).__init__()
        self.gate_fn = gate_fn
        reduced_chs = _make_divisible((reduced_base_chs or in_chs) * se_ratio, divisor)
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.conv_reduce = nn.Conv2d(in_chs, reduced_chs, 1, bias=True)
        self.act1 = act_layer(inplace=True)
        self.conv_expand = nn.Conv2d(reduced_chs, in_chs, 1, bias=True)

    def forward(self, x):
        x_se = self.avg_pool(x)
        x_se = self.conv_reduce(x_se)
        x_se = self.act1(x_se)
        x_se = self.conv_expand(x_se)
        x = x * self.gate_fn(x_se)
        return x


class ConvBnAct(nn.Module):
    def __init__(self, in_chs, out_chs, kernel_size,
                 stride=1, act_layer=nn.ReLU):
        super(ConvBnAct, self).__init__()
        self.conv = nn.Conv2d(in_chs, out_chs, kernel_size, stride, kernel_size // 2, bias=False)
        self.bn1 = nn.BatchNorm2d(out_chs)
        self.act1 = act_layer(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn1(x)
        x = self.act1(x)
        return x


class GhostModule(nn.Module):
    def __init__(self, inp, oup, kernel_size=1, ratio=2, dw_size=3, stride=1, relu=True):
        super(GhostModule, self).__init__()
        self.oup = oup
        init_channels = math.ceil(oup / ratio)
        new_channels = init_channels * (ratio - 1)

        self.primary_conv = nn.Sequential(
            nn.Conv2d(inp, init_channels, kernel_size, stride, kernel_size // 2, bias=False),
            nn.BatchNorm2d(init_channels),
            nn.ReLU(inplace=True) if relu else nn.Sequential(),
        )

        self.cheap_operation = nn.Sequential(
            nn.Conv2d(init_channels, new_channels, dw_size, 1, dw_size // 2, groups=init_channels, bias=False),
            nn.BatchNorm2d(new_channels),
            nn.ReLU(inplace=True) if relu else nn.Sequential(),
        )

    def forward(self, x):
        x1 = self.primary_conv(x)
        x2 = self.cheap_operation(x1)
        out = torch.cat([x1, x2], dim=1)
        return out[:, :self.oup, :, :]



class LambdaConv(nn.Module):
    def __init__(self, in_channels, out_channels, heads=4, k=16, u=1, m=23):
        super(LambdaConv, self).__init__()
        self.kk, self.uu, self.vv, self.mm, self.heads = k, u, out_channels // heads, m, heads
        self.local_context = True if m > 0 else False
        self.padding = (m - 1) // 2

        self.queries = nn.Sequential(
            nn.Conv2d(in_channels, k * heads, kernel_size=1, bias=False),
            nn.BatchNorm2d(k * heads)
        )
        self.keys = nn.Sequential(
            nn.Conv2d(in_channels, k * u, kernel_size=1, bias=False),
        )
        self.values = nn.Sequential(
            nn.Conv2d(in_channels, self.vv * u, kernel_size=1, bias=False),
            nn.BatchNorm2d(self.vv * u)
        )

        self.softmax = nn.Softmax(dim=-1)

        if self.local_context:
            self.embedding = nn.Parameter(torch.randn([self.kk, self.uu, 1, m, m]), requires_grad=True)
        else:
            self.embedding = nn.Parameter(torch.randn([self.kk, self.uu]), requires_grad=True)

    def forward(self, x):
        n_batch, C, w, h = x.size()

        queries = self.queries(x).view(n_batch, self.heads, self.kk, w * h) # b, heads, k // heads, w * h
        softmax = self.softmax(self.keys(x).view(n_batch, self.kk, self.uu, w * h)) # b, k, uu, w * h
        values = self.values(x).view(n_batch, self.vv, self.uu, w * h) # b, v, uu, w * h

        lambda_c = torch.einsum('bkum,bvum->bkv', softmax, values)
        y_c = torch.einsum('bhkn,bkv->bhvn', queries, lambda_c)

        if self.local_context:
            values = values.view(n_batch, self.uu, -1, w, h)
            lambda_p = F.conv3d(values, self.embedding, padding=(0, self.padding, self.padding))
            lambda_p = lambda_p.view(n_batch, self.kk, self.vv, w * h)
            y_p = torch.einsum('bhkn,bkvn->bhvn', queries, lambda_p)
        else:
            lambda_p = torch.einsum('ku,bvun->bkvn', self.embedding, values)
            y_p = torch.einsum('bhkn,bkvn->bhvn', queries, lambda_p)

        out = y_c + y_p
        out = out.contiguous().view(n_batch, -1, w, h)

        return out


class LambdaBottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(LambdaBottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)

        self.conv2 = nn.ModuleList([LambdaConv(planes, planes)])
        if stride != 1 or in_planes != self.expansion * planes:
            self.conv2.append(nn.AvgPool2d(kernel_size=(3, 3), stride=stride, padding=(1, 1)))
        self.conv2.append(nn.BatchNorm2d(planes))
        self.conv2.append(nn.ReLU())
        self.conv2 = nn.Sequential(*self.conv2)

        self.conv3 = nn.Conv2d(planes, self.expansion * planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion * planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.conv2(out)
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=1000):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, 64, num_blocks[0])
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=1)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Sequential(
            nn.Dropout(0.3), # All architecture deeper than ResNet-200 dropout_rate: 0.2
            nn.Linear(512 * block.expansion, num_classes)
        )

    def _make_layer(self, block, planes, num_blocks, stride=1):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for idx, stride in enumerate(strides):
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.maxpool(out)

        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        return out

def LambdaResNet50():
    return ResNet(LambdaBottleneck, [3, 4, 6, 3])


class GeneralizedMeanPooling(nn.Module):
    r"""Applies a 2D power-average adaptive pooling over an input signal composed of several input planes.
    The function computed is: :math:`f(X) = pow(sum(pow(X, p)), 1/p)`
        - At p = infinity, one gets Max Pooling
        - At p = 1, one gets Average Pooling
    The output is of size H x W, for any input size.
    The number of output features is equal to the number of input planes.
    Args:
        output_size: the target output size of the image of the form H x W.
                     Can be a tuple (H, W) or a single H for a square image H x H
                     H and W can be either a ``int``, or ``None`` which means the size will
                     be the same as that of the input.
    """

    def __init__(self, norm, output_size=1, eps=1e-6):
        super(GeneralizedMeanPooling, self).__init__()
        assert norm > 0
        self.p = float(norm)
        self.output_size = output_size
        self.eps = eps

    def forward(self, x):
        x = x.clamp(min=self.eps).pow(self.p)
        return torch.nn.functional.adaptive_avg_pool2d(x, self.output_size).pow(1. / self.p)

    def __repr__(self):
        return self.__class__.__name__ + '(' \
               + str(self.p) + ', ' \
               + 'output_size=' + str(self.output_size) + ')'


class GeneralizedMeanPoolingP(GeneralizedMeanPooling):
    """ Same, but norm is trainable
    """

    def __init__(self, norm=3, output_size=1, eps=1e-6):
        super(GeneralizedMeanPoolingP, self).__init__(norm, output_size, eps)
        self.p = nn.Parameter(torch.ones(1) * norm)

class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)
def build_pool_conv(input_dim=2048,reduce_dim=512):
    pool_reduce = nn.Sequential(
        GeneralizedMeanPoolingP(),
        nn.Conv2d(input_dim, reduce_dim, 1, bias=False),
        nn.BatchNorm2d(reduce_dim),
        nn.ReLU(True),
        Flatten()
    )
    pool_reduce.apply(weights_init_kaiming)
    return pool_reduce



def build_reduction_layer(input_dim,feat_dim):
    fc=nn.Linear(input_dim,feat_dim)
    bn=nn.BatchNorm1d(feat_dim)
    relu=nn.ReLU()
    return nn.Sequential(
        fc,
        bn,
        relu
    )


class ModelWrapper(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model
        self.adaptive_param = self._make_layer(self.model)

    def _make_layer(self, model):
        layers = []

        for _name, _params in self.model.named_parameters():
            if "conv" in _name:
                if "weight" in _name:
                    layers.append(nn.Parameter(data=torch.tensor(0.0)))
                if "bias" in _name:
                    layers.append(nn.Parameter(data=torch.tensor(0.0)))
            if "bn" in _name:
                if "weight" in _name:
                    layers.append(nn.Parameter(data=torch.tensor(0.0)))
                if "bias" in _name:
                    layers.append(nn.Parameter(data=torch.tensor(0.0)))
        return nn.ParameterList(layers)

    def forward(self, images,heads,masks,labels):
        x = self.model(images,heads,masks,labels)
        return x

##主要函数

###shuffle

In [18]:

class Shuffle_block(nn.Module):
    def __init__(self,inchannel,out_channel,repeat_count):
        super(Shuffle_block, self).__init__()
        layers=[]
        layers.append(Shuffle_B(inchannel,inchannel))
        for i in range(repeat_count):
            layers.append(Shuffle_A)
        layers.append(Shuffle_B(inchannel,out_channel))
        self.model=nn.Sequential(*layers)
    def forward(self,x):
        return self.model(x)



###hacnn

In [19]:


class HACNN(nn.Module):
    """Harmonious Attention Convolutional Neural Network.
    Reference:
        Li et al. Harmonious Attention Network for Person Re-identification. CVPR 2018.
    Public keys:
        - ``hacnn``: HACNN.
    """

    # Args:
    #    num_classes (int): number of classes to predict
    #    nchannels (list): number of channels AFTER concatenation
    #    feat_dim (int): feature dimension for a single stream
    #    learn_region (bool): whether to learn region features (i.e. local branch)

    def __init__(
            self,
            loss='softmax',
            nchannels=[128, 256, 384],
            feat_dim=2048,
            use_gpu=True,
            **kwargs
    ):
        super(HACNN, self).__init__()
        self.loss = loss

        self.use_gpu = use_gpu

        self.conv = ConvBlock(3, 32, 3, s=2, p=1)

        # Construct Inception + HarmAttn blocks
        # ============== Block 1 ==============
        self.inception1 = nn.Sequential(
            InceptionA(32, nchannels[0]),
            InceptionB(nchannels[0], nchannels[0]),
        )
        self.ha1 = HarmAttn(nchannels[0])

        # ============== Block 2 ==============
        self.inception2 = nn.Sequential(
            InceptionA(nchannels[0], nchannels[1]),
            InceptionB(nchannels[1], nchannels[1]),
        )
        self.ha2 = HarmAttn(nchannels[1])

        # ============== Block 3 ==============
        self.inception3 = nn.Sequential(
            InceptionA(nchannels[1], nchannels[2]),
            InceptionB(nchannels[2], nchannels[2]),
        )
        self.ha3 = HarmAttn(nchannels[2])

        self.conv_block=nn.Sequential(
            nn.Conv2d(nchannels[2],2048,kernel_size=1),
            nn.BatchNorm2d(2048),
            nn.ReLU()
        )
        self.feat_dim = feat_dim

    def init_scale_factors(self):
        # initialize scale factors (s_w, s_h) for four regions
        self.scale_factors = []
        self.scale_factors.append(
            torch.tensor([[1, 0], [0, 0.25]], dtype=torch.float)
        )
        self.scale_factors.append(
            torch.tensor([[1, 0], [0, 0.25]], dtype=torch.float)
        )
        self.scale_factors.append(
            torch.tensor([[1, 0], [0, 0.25]], dtype=torch.float)
        )
        self.scale_factors.append(
            torch.tensor([[1, 0], [0, 0.25]], dtype=torch.float)
        )

    def stn(self, x, theta):
        """Performs spatial transform

        x: (batch, channel, height, width)
        theta: (batch, 2, 3)
        """
        grid = F.affine_grid(theta, x.size())
        x = F.grid_sample(x, grid)
        return x

    def transform_theta(self, theta_i, region_idx):
        """Transforms theta to include (s_w, s_h), resulting in (batch, 2, 3)"""
        scale_factors = self.scale_factors[region_idx]
        theta = torch.zeros(theta_i.size(0), 2, 3)
        theta[:, :, :2] = scale_factors
        theta[:, :, -1] = theta_i
        if self.use_gpu:
            theta = theta.cuda()
        return theta

    def forward(self, x):
        assert x.size(2) == 160 and x.size(3) == 64, \
            'Input size does not match, expected (160, 64) but got ({}, {})'.format(x.size(2), x.size(3))
        x = self.conv(x)
        x1 = self.inception1(x)
        x1_attn, x1_theta = self.ha1(x1)
        x1_out = x1 * x1_attn
        x2 = self.inception2(x1_out)
        x2_attn, x2_theta = self.ha2(x2)
        x2_out = x2 * x2_attn
        x3 = self.inception3(x2_out)
        x3_attn, x3_theta = self.ha3(x3)
        x3_out = x3 * x3_attn
        feat=self.conv_block(x3_out)
        return feat


###ghost

In [46]:

class GhostBottleneck(nn.Module):
    """ Ghost bottleneck w/ optional SE"""

    def __init__(self, in_chs, mid_chs, out_chs, dw_kernel_size=3,
                 stride=1, act_layer=nn.ReLU, se_ratio=0.):
        super(GhostBottleneck, self).__init__()
        has_se = se_ratio is not None and se_ratio > 0.
        self.stride = stride

        # Point-wise expansion
        self.ghost1 = GhostModule(in_chs, mid_chs, relu=True)

        # Depth-wise convolution
        if self.stride > 1:
            self.conv_dw = nn.Conv2d(mid_chs, mid_chs, dw_kernel_size, stride=stride,
                                     padding=(dw_kernel_size - 1) // 2,
                                     groups=mid_chs, bias=False)
            self.bn_dw = nn.BatchNorm2d(mid_chs)

        # Squeeze-and-excitation
        if has_se:
            self.se = SqueezeExcite(mid_chs, se_ratio=se_ratio)
        else:
            self.se = None

        # Point-wise linear projection
        self.ghost2 = GhostModule(mid_chs, out_chs, relu=False)

        # shortcut
        if (in_chs == out_chs and self.stride == 1):
            self.shortcut = nn.Sequential()
        else:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_chs, in_chs, dw_kernel_size, stride=stride,
                          padding=(dw_kernel_size - 1) // 2, groups=in_chs, bias=False),
                nn.BatchNorm2d(in_chs),
                nn.Conv2d(in_chs, out_chs, 1, stride=1, padding=0, bias=False),
                nn.BatchNorm2d(out_chs),
            )

    def forward(self, x):
        residual = x

        # 1st ghost bottleneck
        x = self.ghost1(x)

        # Depth-wise convolution
        if self.stride > 1:
            x = self.conv_dw(x)
            x = self.bn_dw(x)

        # Squeeze-and-excitation
        if self.se is not None:
            x = self.se(x)

        # 2nd ghost bottleneck
        x = self.ghost2(x)

        x += self.shortcut(residual)
        return x


class GhostNet(nn.Module):
    def __init__(self, cfgs, num_classes=1000, width=1.0, dropout=0.2):
        super(GhostNet, self).__init__()
        # setting of inverted residual blocks
        self.cfgs = cfgs
        self.dropout = dropout

        # building first layer
        output_channel = _make_divisible(16 * width, 4)
        self.conv_stem = nn.Conv2d(3, output_channel, 3, 2, 1, bias=False)
        self.bn1 = nn.BatchNorm2d(output_channel)
        self.act1 = nn.ReLU(inplace=True)
        input_channel = output_channel

        # building inverted residual blocks
        stages = []
        block = GhostBottleneck
        for cfg in self.cfgs:
            layers = []
            for k, exp_size, c, se_ratio, s in cfg:
                output_channel = _make_divisible(c * width, 4)
                hidden_channel = _make_divisible(exp_size * width, 4)
                layers.append(block(input_channel, hidden_channel, output_channel, k, s,
                                    se_ratio=se_ratio))
                input_channel = output_channel
            stages.append(nn.Sequential(*layers))

        output_channel = _make_divisible(exp_size * width, 4)
        stages.append(nn.Sequential(ConvBnAct(input_channel, output_channel, 1)))
        input_channel = output_channel

        self.blocks = nn.Sequential(*stages)
        self.conv_final=nn.Sequential(
            nn.Conv2d(960,2048,kernel_size=1),
            nn.BatchNorm2d(2048),
            nn.ReLU()
        )
        # building last several layers
        output_channel = 1280


    def forward(self, x):
        x = self.conv_stem(x)
        x = self.bn1(x)
        x = self.act1(x)
        x = self.blocks(x)
        x=self.conv_final(x)

        return x

def ghostnet(**kwargs):
    """
    Constructs a GhostNet model
    """
    cfgs = [
        # k, t, c, SE, s
        # stage1
        [[3,  16,  16, 0, 1]],
        # stage2
        [[3,  48,  24, 0, 2]],
        [[3,  72,  24, 0, 1]],
        # stage3
        [[5,  72,  40, 0.25, 2]],
        [[5, 120,  40, 0.25, 1]],
        # stage4
        [[3, 240,  80, 0, 2]],
        [[3, 200,  80, 0, 1],
         [3, 184,  80, 0, 1],
         [3, 184,  80, 0, 1],
         [3, 480, 112, 0.25, 1],
         [3, 672, 112, 0.25, 1]
        ],
        # stage5
        [[5, 672, 160, 0.25, 2]],
        [[5, 960, 160, 0, 1],
         [5, 960, 160, 0.25, 1],
         [5, 960, 160, 0, 1],
         [5, 960, 160, 0.25, 1]
        ]
    ]
    return GhostNet(cfgs, **kwargs)

###backbone

In [47]:


from torchvision.models import resnet50
from geffnet import efficientnet_b0
def build_backbone(model_name):
    if model_name=='resnet50':
        model = resnet50(True)
        stride = 1
        model.layer4[0].downsample[0].stride = stride
        model.layer4[0].conv2.stride = stride
        base = nn.Sequential(
            model.conv1,
            model.bn1,
            model.maxpool,
            model.layer1,
            model.layer2,
            model.layer3,
            model.layer4
        )
        feat=2048
    elif model_name=='ghost':
        base=ghostnet()
        feat=2048
    elif model_name=='Lambda':
        base=LambdaResNet50()
        feat=2048
    elif model_name=='hacnn':
        base=HACNN()
        feat=2048


    return base,feat


#head branch

In [81]:

def conv_block(inchannel,outchannel):
    return nn.Sequential(
        nn.Conv2d(inchannel,outchannel,kernel_size=1),
        nn.BatchNorm2d(outchannel),
        nn.ReLU()
    )



class Head_Branch(nn.Module):
    def __init__(self,cfg,num_class):
        super(Head_Branch,self).__init__()
        self.backbone,feat_dim=build_backbone(cfg['head_backbone'])
        self.cfg=cfg
        self.cls_criterion=CrossEntropyLabelSmoothLoss(num_class)
        self.circle_criterion=CircleLoss(m=0.25, gamma=80)
        self.flat=Flatten()
        if cfg['head_attention']:
            if cfg['HAN']:
                self.has_1_pool=build_pool_conv(feat_dim,512)
                self.has_2_pool=build_pool_conv(feat_dim,512)
                self.has_3_pool=build_pool_conv(feat_dim,512)

                self.has_red=build_reduction_layer(512*3,1024)
                self.head_bn = nn.BatchNorm1d(1024)
                self.head_bn.bias.requires_grad_(False)
                self.head_classfier = nn.Linear(1024, num_class, bias=False)
                self.head_bn.apply(weights_init_kaiming)
                self.head_classfier.apply(weights_init_classifier)

                self.han_pool_1 = GeneralizedMeanPoolingP()
                self.han_c_att_1 = nn.Sequential(nn.Linear(2048, 1024, bias=False), nn.ReLU(),
                                                 nn.Linear(1024, 2048, bias=False))
                
                self.han_c_att_1[0].apply(weights_init_classifier)
                self.han_c_att_1[1].apply(weights_init_classifier)
                self.han_pool_2 = GeneralizedMeanPoolingP()
                self.han_c_att_2 = nn.Sequential(nn.Linear(2048, 1024, bias=False), nn.ReLU(),
                                                 nn.Linear(1024, 2048, bias=False))
                self.han_c_att_2[0].apply(weights_init_classifier)
                self.han_c_att_2[1].apply(weights_init_classifier)
                self.han_pool_3 = GeneralizedMeanPoolingP()
                self.han_c_att_3 = nn.Sequential(nn.Linear(2048, 1024, bias=False), nn.ReLU(),
                                                 nn.Linear(1024, 2048, bias=False))
                self.han_c_att_3[0].apply(weights_init_classifier)
                self.han_c_att_3[1].apply(weights_init_classifier)

            elif cfg['BCOS']:
                self.branch1_conv=conv_block(feat_dim,512)
                self.branch1_pool=GeneralizedMeanPoolingP()
                self.branch1_bn=nn.BatchNorm1d(512)
                self.branch1_bn.bias.requires_grad_(False)
                self.branch1_fc=nn.Linear(512,num_class,bias=False)
                self.branch1_bn.apply(weights_init_kaiming)
                self.branch1_fc.apply(weights_init_classifier)


                self.branch4_conv=conv_block(2048,1024)
                self.branch4_pool=nn.ModuleList()
                self.branch4_pool_conv=nn.ModuleList()
                for i in range(3):
                    self.branch4_pool.append(GeneralizedMeanPoolingP())
                    self.branch4_pool_conv.append(conv_block(1024,512))


                self.branch4_bn = nn.BatchNorm1d(512*3)
                self.branch4_bn.bias.requires_grad_(False)
                self.branch4_fc = nn.Linear(512*3, num_class, bias=False)
                self.branch4_bn.apply(weights_init_kaiming)
                self.branch4_fc.apply(weights_init_classifier)


                self.branch2_conv=conv_block(2048,2048)
                self.local_3_conv_list = nn.ModuleList()
                self.rest_3_conv_list = nn.ModuleList()
                self.relation_3_conv_list = nn.ModuleList()
                for i in range(3):
                    self.local_3_conv_list.append(nn.Sequential(
                        nn.Conv2d(2048, 256, 1),
                        nn.BatchNorm2d(256),
                        nn.ReLU(inplace=True)))
                for i in range(3):
                    self.rest_3_conv_list.append(nn.Sequential(
                        nn.Conv2d(2048, 256, 1),
                        nn.BatchNorm2d(256),
                        nn.ReLU(inplace=True)))
                for i in range(3):
                    self.relation_3_conv_list.append(nn.Sequential(
                        nn.Conv2d(256 * 2, 256, 1),
                        nn.BatchNorm2d(256),
                        nn.ReLU(inplace=True)))
                self.fc_local_rest_3_list = nn.ModuleList()
                for _ in range(3):
                    fc = nn.Linear(256, num_class,bias=False)
                    fc.apply(weights_init_classifier)
                    self.fc_local_rest_3_list.append(fc)

                self.branch3_conv=conv_block(2048,2048)
                self.global_3_max_conv_list=nn.ModuleList()
                self.global_3_rest_conv_list=nn.ModuleList()
                self.global_3_pooling_conv_list=nn.ModuleList()
                self.global_3_max_conv_list.append(nn.Sequential(
                    nn.Conv2d(2048, 256, 1),
                    nn.BatchNorm2d(256),
                    nn.ReLU(inplace=True)))
                self.global_3_rest_conv_list.append(nn.Sequential(
                    nn.Conv2d(2048, 256, 1),
                    nn.BatchNorm2d(256),
                    nn.ReLU(inplace=True)))
                self.global_3_pooling_conv_list.append(nn.Sequential(
                    nn.Conv2d(512, 256, 1),
                    nn.BatchNorm2d(256),
                    nn.ReLU(inplace=True)))
                self.fc_global_rest_3_list = nn.ModuleList()
                fc = nn.Linear(256, num_class,bias=False)
                fc.apply(weights_init_classifier)
                self.fc_global_rest_3_list.append(fc)





        else:
            self.pool=GeneralizedMeanPoolingP()
            self.bn=nn.BatchNorm1d(feat_dim)
            self.bn.bias.requires_grad_(False)
            self.fc=nn.Linear(feat_dim,num_class,bias=False)
            self.bn.apply(weights_init_kaiming)
            self.fc.apply(weights_init_classifier)


    def HAN_1(self, x):
        c_att = self.han_c_att_1(self.han_pool_1(x).view(x.shape[0], -1))
        c_att = F.sigmoid(c_att).view(x.shape[0], -1, 1, 1)
        feat = x + torch.mul(x, c_att)
        s_att = F.sigmoid(torch.sum(feat, dim=1)).unsqueeze(1)
        han_feat = torch.mul(feat, s_att)
        return han_feat

    def HAN_2(self, x):
        c_att = self.han_c_att_2(self.han_pool_2(x).view(x.shape[0], -1))
        c_att = F.sigmoid(c_att).view(x.shape[0], -1, 1, 1)
        feat = x + torch.mul(x, c_att)
        s_att = F.sigmoid(torch.sum(feat, dim=1)).unsqueeze(1)
        han_feat = torch.mul(feat, s_att)
        return han_feat

    def HAN_3(self, x):
        c_att = self.han_c_att_3(self.han_pool_3(x).view(x.shape[0], -1))
        c_att = F.sigmoid(c_att).view(x.shape[0], -1, 1, 1)
        feat = x + torch.mul(x, c_att)
        s_att = F.sigmoid(torch.sum(feat, dim=1)).unsqueeze(1)
        han_feat = torch.mul(feat, s_att)
        return han_feat
    def forward(self,x,label):
        x=self.backbone(x)
        loss=0
        if self.cfg['head_attention']:
            if self.cfg['HAN']:
                height=int(x.shape[2]//3)
                hf_1=self.HAN_1(x[:,:,0*height:1*height,:])
                hf_1=self.has_1_pool(hf_1)
                

                hf_2=self.HAN_2(x[:,:,1*height:2*height,:])
                hf_2=self.has_2_pool(hf_2)
                
                hf_3=self.HAN_3(x[:,:,2*height:3*height,:])
                hf_3=self.has_3_pool(hf_3)
                
                hf=torch.cat([hf_1,hf_2,hf_3],dim=1)
                global_feat=self.has_red(hf)
                feat=self.head_bn(global_feat)
                logit=self.head_classfier(feat)

                if self.training:
                  loss=loss+self.circle_criterion(hf_3,label)
                  loss=loss+self.circle_criterion(hf_2,label)
                  loss=loss+self.circle_criterion(hf_1,label)
                  loss=loss+self.circle_criterion(global_feat,label)
                  loss=loss+self.cls_criterion(logit,label)
            elif self.cfg['BCOS']:
                BCOS_feat=[]
                branch1_feat=self.branch1_conv(x)
                branch1_feat=self.branch1_pool(branch1_feat)
                branch1_feat=self.flat(branch1_feat)
                feat1=self.branch1_bn(branch1_feat)
                logit1=self.branch1_fc(feat1)
                BCOS_feat.append(feat1)

                branch2_feat=self.branch2_conv(x)
                stripe_h_3 = int(branch2_feat.size(2) / 3)
                local_feat_list=[]
                final_feat_list=[]
                rest_feat_list=[]

                local_rest_logit=[]
                for i in range(3):
                    local_3_feat = F.max_pool2d(
                        branch2_feat[:, :, i * stripe_h_3: (i + 1) * stripe_h_3, :],
                        (stripe_h_3, branch2_feat.size(-1)))
                    local_feat_list.append(local_3_feat)
                for i in range(3):
                    rest_feat_list.append((local_feat_list[(i + 1) % 3]
                                             + local_feat_list[(i + 2) % 3]
                                           )/2)


                for i in range(3):
                    local_6_feat = self.local_3_conv_list[i](local_feat_list[i]).squeeze(3).squeeze(2)
                    input_rest_6_feat = self.rest_3_conv_list[i](rest_feat_list[i]).squeeze(3).squeeze(2)

                    input_local_rest_6_feat = torch.cat((local_6_feat, input_rest_6_feat), 1).unsqueeze(2).unsqueeze(3)

                    local_rest_6_feat = self.relation_3_conv_list[i](input_local_rest_6_feat)

                    local_rest_6_feat = (local_rest_6_feat
                                         + local_6_feat.unsqueeze(2).unsqueeze(3)).squeeze(3).squeeze(2)

                    final_feat_list.append(local_rest_6_feat)

                    local_rest_logit.append(self.fc_local_rest_3_list[i](local_rest_6_feat))



                global_feat_list=[]
                global_logit_list=[]

                branch3_feat=self.branch3_conv(x)
                for i in range(3):
                    local_3_feat = F.max_pool2d(
                        branch2_feat[:, :, i * stripe_h_3: (i + 1) * stripe_h_3, :],
                        (stripe_h_3, branch2_feat.size(-1)))
                    global_feat_list.append(local_3_feat)
                global_2_max_feat = F.max_pool2d(branch3_feat, (branch3_feat.size(2), branch3_feat.size(3)))
                global_2_rest_feat = (global_feat_list[0] + global_feat_list[1] - global_2_max_feat)
                global_2_max_feat = self.global_3_max_conv_list[0](global_2_max_feat)
                global_2_rest_feat = self.global_3_rest_conv_list[0](global_2_rest_feat)
                global_2_max_rest_feat = self.global_3_pooling_conv_list[0](
                    torch.cat((global_2_max_feat, global_2_rest_feat), 1))
                global_2_feat = (global_2_max_feat + global_2_max_rest_feat).squeeze(3).squeeze(2)
                final_feat_list.append(global_2_feat)
                global_logit_list.append(self.fc_global_rest_3_list[0](global_2_feat))

                branch4_feat=self.branch4_conv(x)
                stride=int(branch4_feat.shape[2]//3)
                branch4_feat_list=[]
                for i in range(3):
                    stride_feat=branch4_feat[:,:,i*stride:(i+1)*stride,:]
                    branch4_local_feat=self.branch4_pool[i](stride_feat)
                    branch4_local_feat=self.branch4_pool_conv[i](branch4_local_feat)
                    branch4_local_feat=self.flat(branch4_local_feat)
                    branch4_feat_list.append(branch4_local_feat)
                branch4_feat=torch.cat(branch4_feat_list,dim=1)
                feat4=self.branch4_bn(branch4_feat)
                BCOS_feat.append(feat4)
                logit4=self.branch4_fc(feat4)

                

                if self.training:
                  for feat in final_feat_list:
                      loss=loss+self.circle_criterion(feat,label)

                  for logit in global_logit_list:
                      loss=loss+self.cls_criterion(logit,label)

                  for logit in local_rest_logit:
                      loss=loss+self.cls_criterion(logit,label)


                  loss=loss+self.cls_criterion(logit1,label)
                  loss=loss+self.circle_criterion(feat1,label)

                  loss=loss+self.cls_criterion(logit4,label)
                  loss=loss+self.circle_criterion(feat4,label)
                BCOS_feat.extend(final_feat_list)
                feat=torch.cat(BCOS_feat,dim=1)
                


        else:
            global_feat=self.pool(x)
            global_feat=self.flat(global_feat)
            feat=self.bn(global_feat)
            logit=self.fc(feat)
            if self.training:
              loss=loss+self.circle_criterion(global_feat,label)
              loss=loss+self.cls_criterion(logit,label)
        return feat,loss


# global_branch

In [23]:


class Global_Branch(nn.Module):
    def __init__(self,cfg,num_class):
        self.backbone,feat_dim=build_backbone(cfg['global_backbone'])
        self.pool=GeneralizedMeanPoolingP()
        self.bn=nn.BatchNorm1d(self.in_planes)
        self.bn.bias.requires_grad_(False)
        self.bn.apply(weights_init_kaiming)
        self.fc=nn.Linear(feat_dim,num_class)
        self.fc.apply(weights_init_classifier)
        self.cls_criterion=CrossEntropyLabelSmoothLoss(num_class)
        self.circle_criterion=CircleLoss(m=0.25,gamma=80)

    def forward(self,x,label):
        global_feat = self.pool(self.backbone(x))
        global_feat = global_feat.view(global_feat.shape[0], -1)
        feat=self.bn(global_feat)
        logit=self.fc(feat)
        loss=0
        if self.training:
          loss=loss+self.cls_criterion(logit,label)
          loss=loss+self.circle_criterion(global_feat,label)
        return feat,loss


# mask_branch

In [104]:


import math
class Mask_Branch(nn.Module):
    def __init__(self,cfg,num_class):
        super(Mask_Branch, self).__init__()
        self.cls_criterion=CrossEntropyLabelSmoothLoss(num_class)
        self.circle_criterion=CircleLoss(m=0.25,gamma=80)
        self.N=cfg['N']
        self.M=cfg['M']
        self.lambdak=nn.Parameter(torch.from_numpy(np.array([0.05]*self.M)))
        self.a0=-1*math.pi
        self.b0=1*math.pi
        self.a1=-1*math.pi/4
        self.b1=-3*math.pi/4
        self.a2=3*math.pi/4
        self.b2=math.pi/4
        self.R_max=2
        self.backbone_list=nn.ModuleList()
        for i in range(3):
            backbone,feat_dim=build_backbone(cfg['mask_backbone'])
            self.backbone_list.append(backbone)

        self.pool_list=nn.ModuleList()
        for i in range(3):
            for j in range(4):
                pool=GeneralizedMeanPoolingP()
                self.pool_list.append(pool)

        self.ASE_FC_list=nn.ModuleList()
        self.ASE_CONV_list=nn.ModuleList()
        for i in range(3):
            for j in range(4):
                fc_block=nn.Sequential(
                    nn.Linear(2048,512),
                    nn.ReLU(),
                    nn.Linear(512,2048),
                )
                conv_b=nn.Sequential(
                    nn.Conv2d(2048,256,kernel_size=1),
                    nn.BatchNorm2d(256),
                    nn.ReLU()
                )
                self.ASE_FC_list.append(fc_block)
                self.ASE_CONV_list.append(conv_b)

        self.fc_list=nn.ModuleList()
        for i in range(3):
            for j in range(4):
                fc=nn.Linear(256,num_class)
                self.fc_list.append(fc)

    def ASE(self,x,index):
        feat=self.ASE_FC_list[index](x.view(x.shape[0],-1))
        x=x+torch.mul(x,F.sigmoid(feat).view(x.shape[0],-1,1,1))
        x=self.ASE_CONV_list[index](x)
        return x


    def forward(self,x,label):
        theta_list0 = []
        theta_list1 = []
        theta_list2 = []
        for i in range(self.M):
            zi = self.lambdak[:i].sum() / self.lambdak.sum()
            fi = (self.b0 - self.a0) * zi + self.a0
            theta_list0.append(fi)
        for i in range(self.M):
            zi = self.lambdak[:i].sum() / self.lambdak.sum()
            fi = (self.b1 - self.a1) * zi + self.a1
            theta_list1.append(fi)
        for i in range(self.M):
            zi = self.lambdak[:i].sum() / self.lambdak.sum()
            fi = (self.b2 - self.a2) * zi + self.a2
            theta_list2.append(fi)
        theta_lists=np.zeros((3,self.M))
        theta_lists[0]=theta_list0
        theta_lists[1]=theta_list1
        theta_lists[2]=theta_list2
        theta_lists=torch.from_numpy(theta_lists)
        self.grid = torch.zeros((3, 1, self.N, self.M, 2))
        for k in range(3):
            for i in range(self.N):
                for j in range(self.M):
                    self.grid[k,0,i,j,0]=self.R_max*j*torch.cos(theta_lists[k,i])/self.M
                    self.grid[k,0,i,j,1]=self.R_max*j*torch.sin(theta_lists[k,i])/self.M
        self.grid = self.grid.cuda()

        grid_features=[]
        for i in range(3):
            grid=self.grid[i]
            grid=grid.repeat(x.shape[0],1,1,1)
            grid_feature=F.grid_sample(x,grid)
            grid_features.append(grid_feature)
        loss=0
        logit_list=[]
        final_feature_list=[]
        for i in range(3):
            features=self.backbone_list[i](grid_features[i])
            stride=int(features.shape[2]/4)
            for j in range(4):
                feature=features[:,:,j*stride:(j+1)*stride,:]
                feature=self.pool_list[i*4+j](feature)
                feature=self.ASE(feature,i*4+j)
                feature=feature.view(feature.shape[0],-1)
                logit=self.fc_list[i*4+j](feature)
                logit_list.append(logit)
                final_feature_list.append(feature)
        feat=torch.cat(final_feature_list,dim=1)
        if self.training:        
          for logit in logit_list:
              loss=loss+self.cls_criterion(logit,label)

          
          loss=loss+self.circle_criterion(feat,label)
        else:
          loss=0
        return feat,loss


#final_model

In [107]:

class myModel(nn.Module):
    def __init__(self,cfg,num_class):
        super(myModel,self).__init__()
        self.global_branch=Global_Branch(cfg,num_class)
        global_feat=2048
        self.head_branch=Head_Branch(cfg,num_class)
        if cfg['head_attention']:
          if cfg['HAN']:
            head_feat=1024
          elif cfg['BCOS']:
            head_feat=3072
        else:
          head_feat=2048

        self.mask_branch=Mask_Branch(cfg,num_class)
        mask_feat=3072
        self.cls_criterion = CrossEntropyLabelSmoothLoss()
        self.circle_criterion = CircleLoss(m=0.25, gamma=80)

        all_feat=global_feat+head_feat+mask_feat
        self.reduce_linear=nn.Linear(4096,2048)
        self.reduction_block=nn.Sequential(
            nn.Conv2d(all_feat,4096,kernel_size=1),
            nn.BatchNorm2d(4096),
            nn.ReLU(),
            nn.Dropout()
        )
        self.reduction_bn=nn.BatchNorm1d(2048)
        self.reduction_bn.bias.requires_grad_(False)
        self.final_attention=nn.Sequential(
            nn.Linear(2048,2048//16),
            nn.ReLU(),
            nn.Linear(2048//16,2048),
            nn.Sigmoid()
        )
        self.final_cls=nn.Linear(2048,num_class,bias=False)
        super(myModel, self).__init__()

    def forward(self,images,heads,masks,label):
        out={}
        global_feature,global_loss=self.global_branch(images)
        head_feature,head_loss=self.head_branch(heads)
        mask_feature,mask_loss=self.mask_branch(masks)
        all_feature=torch.cat([global_feature,head_feature,mask_feature],dim=1)
        batch=all_feature.shape[0]
        all_feature=all_feature.reshape(batch,-1,1,1)
        all_feature=self.reduction_block(all_feature)
        all_feature=all_feature.squeeze()
        all_feature=self.reduce_linear(all_feature)
        all_feature=self.reduce_bn(all_feature)
        attention_mask=self.final_attention(all_feature)
        all_feature=all_feature*attention_mask
        all_logit=self.final_cls(all_feature)
        if self.training:
          loss=0
          loss=loss+self.cls_criterion(all_logit,label)
          loss=loss+self.circle_criterion(all_feature,label)
        else:
          loss=0
        out['global_loss']=global_loss
        out['head_loss']=head_loss
        out['final_loss']=loss
        out['mask_loss']=mask_loss
        
        out['global_feature']=global_feature
        out['head_feature']=head_feature
        out['mask_feature']=mask_feature
        out['final_feature']=all_feature
        
        return out


In [106]:
device=torch.device('cuda:0')
model=myModel(cfg,100)
images=torch.rand(64,3,384,128)
masks=torch.rand(64,3,384,128)
faces=torch.rand(64,3,224,224)

labels=torch.ones(64).long()

model=model.to(device)
images=images.to(device)
masks=masks.to(device)
faces=faces.to(device)
labels=labels.to(device)
scaler=GradScaler()
with autocast():
  out=model(images,masks,faces,labels)
  


AttributeError: ignored

#训练

In [27]:
from tqdm import tqdm
import logging
import albumentations as albu
import numpy as np
import torch
from iglovikov_helper_functions.utils.image_utils import load_rgb, pad, unpad
from iglovikov_helper_functions.dl.pytorch.utils import tensor_from_rgb_image
scaler=GradScaler()
class Trainer:
  def __init__(self,cfg,train_component):
    self.train_loaders=train_component['train_loaders']
    self.val_loader=train_component['val_loader']
    self.reid_model=train_component['model']
    self.optimizer=train_component['optimizer']
    self.scheduler=train_component['scheduler']
    self.logger=train_component['logger']
    self.num_class=train_component['num_class']
    self.num_query=train_component['num_query']
    self.checkpoint=train_component['checkpoint']
    self.face_detect_model=train_component['face_model']
    self.mask_detect_model=train_component['mask_model']
    self.face_t=train_component['face_transform']
    if cfg['ema']:
      self.ema=train_component['ema']
    if cfg['l2']:
      self.l2_criterion=train_component['l2_criterion']
    self.device=torch.device('cuda:0')
    self.cfg=cfg


  def train_one_epoch(self,train_loader):
    face_id=0
    for images,origin_images,pids in train_loader:
      loss=0
      self.optimizer.zero_grad()
      heads=[]
      for image in origin_images:
        head=self.face_detect(image)
        heads.append(head)
      heads=torch.cat(heads,dim=0)
      heads=heads.to(device)
      torch.cuda.empty_cache()
      masks=[]
      for image in origin_images:
        mask=self.mask_detect(image)
        masks.append(mask)
      masks=torch.cat(masks,dim=0)
      masks=masks.to(device)
      torch.cuda.empty_cache()
      images=images.to(device)
      pids=pids.to(devide)
      with autocast():
        outs=model(images,heads,masks,pids)
        loss=loss+out['global_loss']
        loss=loss+out['head_loss']
        loss=loss+out['mask_loss']
        loss=loss+out['final_loss']
        if cfg['l2']:
          loss=loss+self.l2_criterion(model,pids)
      scaler.scale(loss).backward()
      scaler.step(optimizer)
      scaler.update() 
      if self.cfg['ema']:
        self.ema.update()
      del images
      del heads
      del masks
    self.scheduler.step()
  
  def train(self):
    epochs=self.cfg['epochs']

    with tqdm(total=epochs) as pbar:
      for epoch in range(epochs):
        for loader in self.train_loaders:
          model.train()
          self.train_one_epoch(loader)
          
          if epoch%5==0:
            if cfg['ema']:
              ema.apply_shadow()
            map0,map1=do_eval(cfg,model,val_loader,num_query)
            checkpoint((map0+map1)/2,model,optimizer)
            if cfg['ema']:
              ema.restore()
        pbar.update(1)
  
  
  def mask_detect(self,image):
    transform=albu.Compose([albu.Normalize(p=1)],p=1)
    padded_image, pads = pad(image, factor=32, border=cv2.BORDER_CONSTANT)
    x = transform(image=padded_image)["image"]
    x = torch.unsqueeze(tensor_from_rgb_image(x), 0)
    x=x.to(self.device)
    x=x.half()
    with torch.no_grad():
      prediction=self.mask_detect_model(x)[0][0]
    mask = (prediction > 0).detach().cpu().numpy().astype(np.uint8)
    contours , hierarchy = cv2.findContours ( mask , cv2.RETR_LIST , cv2.CHAIN_APPROX_SIMPLE )
    new_image=np.zeros((mask.shape[0],mask.shape[1]))
    cv2.drawContours(new_image, contours,-1,(255,0,255),3)
    new_image=torch.from_numpy(new_image)
    new_image=new_image.unsqueeze(0)
    new_image=new_image.repeat(3,1,1)
    new_image=new_image.unsqueeze(0)
    return new_image

  
  def face_detect(self,image):
    image=image.numpy().transpose(1,2,0)
    image=image*255
    boxes,labels,probs=self.face_detect_model.predict(image,1500/2,0.6)
    if boxes.shape[0]==0:
          head=image[:75,120:200]
    else:
          max_index=int(probs.argmax())
          x,y,h,w=boxes[max_index]
          x=int(x)
          y=int(y)
          h=int(h)
          w=int(w)
          miny=max(0,y-20)
          maxy=min(y+w+20,320)
          minx=max(0,x-40)
          maxx=min(x+h+5,320)
          head=image[miny:maxy,minx:maxx]
    if os.path.exit(os.path.join('face','{}.jpg'.format(face_id))):
          pass
    else:
          cv2.imwrite(os.path.join('face','{}.jpg'.format(face_id)),head)
    head=self.face_t(head)
    head=head.unsqueeze(0)
    return head
  
  def eval(self):
    self.reid_model.eval()
    if cfg['eval_all']:
      metric0=MARKET_MAP(num_query,all=True,one_day=True,date_length=date_length)
    else:
      metric0=MARKET_MAP(num_query,all=False,one_day=True,date_length=date_length)
      metric1=MARKET_MAP(num_query,all=False,one_day=False,date_length=date_length)
    with torch.no_grad():
        for imgs,origin_images,pids,cams,dates in self.val_loader:
          heads=[]
          for image in origin_images:
            head=self.face_detect(image)
            heads.append(head)
          heads=torch.cat(heads,dim=0)
          heads=heads.to(device)
          torch.cuda.empty_cache()
          masks=[]
          for image in origin_images:
            mask=self.mask_detect(image)
            masks.append(mask)
          masks=torch.cat(masks,dim=0)
          masks=masks.to(device)

          images=imgs.to(device)    
          out=model(images,heads,masks,pids)
          for out in zip(out['final_feat'],pids,cams,dates):
            metric0.update(out)
            if cfg['eval_all']:
              pass
            else:
              metric1.update(out)

    map0=metric0.compute()
    if cfg['eval_all']:
      return map0,map0
    else:
      map1=metric1.compute()
      return map0,map1       
    

        



      


#主要函数

In [28]:
import sys
import cv2
sys.path.append('/content/Ultra-Light-Fast-Generic-Face-Detector-1MB/vision')
from vision.ssd.config.fd_config import define_img_size
define_img_size(320)
from vision.ssd.mb_tiny_fd import create_mb_tiny_fd, create_mb_tiny_fd_predictor
from vision.ssd.mb_tiny_RFB_fd import create_Mb_Tiny_RFB_fd, create_Mb_Tiny_RFB_fd_predictor
from people_segmentation.pre_trained_models import create_model
from people_segmentation.pre_trained_models import create_model
def main(cfg):
  train_component={}
  train_loaders,val_loader,num_query,num_classes=MAKE_DATALOADER(cfg)
  if cfg['l2']:
    model=ModelWrapper(model)
  model=myModel(cfg,num_classes)
  if cfg['optimizer']=='adam':
    optimizer=torch.optim.Adam(model.parameters(),lr=cfg['lr'],weight_decay=cfg['weight_decay'])
  elif cfg['optimizer']=='sgd':
    optimizer=torch.optim.SGD(model.parameters(),lr=cfg['lr'],momentum=cfg['momentum'],weight_decay=cfg['weight_decay'])
  elif cfg['optimizer']=='rangerlars':
    optimizer=RangerLars(model.parameters(),lr=cfg['lr'])
  
  if cfg['scheduler']=='warmup':

    scheduler = WarmupMultiStepLR(optimizer,cfg['steps'], cfg['gamma'],
                  cfg['warmup_factors'],cfg['warmup_iters'], 
                  'linear')
  elif cfg['scheduler']=='flat':
    scheduler=Flat(optimizer,cfg)
  
  ckpt=cfg['ckpt']
  logpt=cfg['logpt']
  logger=logging.getLogger()
  fh = logging.FileHandler("spam.log")
  fh.setLevel(logging.DEBUG)
  logger.addHandler(fh)
  checkpoint=Checkpoint(ckpt)
  face_model_path=ckpt['face_ckpt']
  class_names = ['face','background']
  face_net = create_Mb_Tiny_RFB_fd(len(class_names), is_test=True, device='cuda:0')

  face_net.load(face_model_path)
  face_predictor = create_Mb_Tiny_RFB_fd_predictor(face_net, candidate_size=1500, device='cuda:0')
  mask_model = create_model("Unet_2020-07-20")
  mask_model.eval()
  face_transform=build_face_transform(cfg)
  train_component['train_loaders']=train_loaders
  train_component['val_loader']=val_loader
  train_component['model']=model
  train_component['optimizer']=optimizer
  train_component['scheduler']=scheduler
  train_component['logger']=logger
  train_component['num_class']=num_classes
  train_component['num_query']=num_query
  train_component['checkpoint']=checkpoint
  train_component['face_model']=face_predictor
  train_component['mask_model']=mask_model
  train_component['face_transform']=face_transform
  if cfg['ema']:
    ema=EMA(model,0.999)
    ema.register()
    train_component['ema']=ema
  if cfg['l2']:
    l2_criterion=Adaptive_l2_loss()
    train_component['l2_criterion']
  trainer=Trainer(cfg,train_component)


  


priors nums:4420


In [29]:
import random
import numpy as np
import torch
np.random.seed(42)
random.seed(42)
torch.backends.cudnn.benchmark=True
torch.manual_seed(42)
torch.cuda.manual_seed(42)       
cfg={
    'train_size':[384,128],
    'test_size':[384,128],
    'train_face_size':[224,224],
    'test_face_size':[224,224],
    'padding':10,
    'mean':[0.485, 0.456, 0.406],
    'std':[0.229, 0.224, 0.225],

    
    'num_workers':8,
    'SAMPLER':'',

    'train_bs':64,
    'test_bs':64,

    'train_K_instances':16,
     
    'epochs':50,
    'ckpt':'reid_test1.pt',
    'logpt':'test1.log',

    
   
    'lr':3e-4,
    'momentum':0.9,
    'weight_decay':5e-4,
     
    'margin':0.3,
    'steps':[35,55],
    'warmup_iters':10,
    'warmup_factors':1/3,
    'gamma':0.1,
    
    
     
    'optimizer':'rangerlars',
    'scheduler':'flat',
    'l2':True,
    'ema':False,
    'mixstyle':True,
     
    'face_ckpt':'/content/drive/MyDrive/dataset/REID/version-RFB-320.pth',
    'N':128,
    'M':128,
    'eval_all':True,
    'global_backbone':'ghost',
    'head_backbone':'ghost',
    'mask_backbone':'ghost',
    'head_attention':True,
    'HAN':False,
    'BCOS':True,
    
     
    'train_data':['market'],
    'val_data':['market'],

}

In [ ]:
data_roots={
    'pdataset':PDataset('/content/datasets/reid1/reid1'),
    'vc':VC_Clothes('/content/datasets/train'),
    'market':Market('/content/datasets')
}

# main(cfg)